### import libraries 

In [2]:
import shutil, os
import numpy as np
import pandas as pd
import math
from math import atan2
import random
import statistics
import csv
from scipy.optimize import minimize

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch
from matplotlib import rcParams, cycler
import matplotlib.gridspec as gridspec
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes,  zoomed_inset_axes
# from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
%matplotlib inline

import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiPolygon, box, mapping
from shapely.ops import nearest_points, unary_union #, cascaded_union
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning, module="shapely")
#import shapely.speedups

import rasterio
from rasterio.mask import mask
from rasterio.plot import show
from rasterio import Affine # or from affine import Affine
from rasterio.plot import plotting_extent
from glob import glob 

import osmnx as ox
import networkx as nx
ox.settings.log_console=True
ox.__version__


'2.0.6'

### read files (eez, eez_12nm, eez_24nm) and initialize parameters

In [4]:
def Param_Initialize():
    global eez, eez_12nm, eez_24nm, IFA, IFA_poly, simulation_type, init_fish, Beta, resource_threshold, low_end  
    global move_fish, growth_prob, K, num_pirogue, catchability, time, time1, frac_coop, coop_effort, noncoop_effort
    global TOTAL_CATCH, TOTAL_FISH, CURRENT_CATCH, AVERAGE_EFFORT, num_days, month, closure_month
    
    # reading shapefiles
    eez  = gpd.read_file(os.path.join(os.getcwd(), 'eez/eez.shp')) # Exclusive Economic Zone (200 nautical miles)
    eez_12nm  = gpd.read_file(os.path.join(os.getcwd(), 'eez_12nm/eez_12nm.shp')) # 12 nautical miles zones (territorial seas)
    eez_24nm  = gpd.read_file(os.path.join(os.getcwd(), 'eez_24nm/eez_24nm.shp')) # 24 nautical miles zones (contiguous zones)
    
    ##reproject crs (to convert from sq. meters to sq. kilometers multiply by 1e-6)
    eez  = eez.to_crs(epsg=32630) 
    eez_12nm  = eez_12nm.to_crs(epsg=32630) 
    eez_24nm  = eez_24nm.to_crs(epsg=32630) 

    # remove inland waters from eez_12nm
    inland_water1=box(840e3, 640e3, 945e3, 680e3)
    inland_water2=box(480e3, 562.25e3, 520e3, 570e3)
    inland_waters = gpd.GeoDataFrame([inland_water1,inland_water2],geometry='geometry',columns=['geometry'], crs=eez_12nm.crs) 
    eez_12nm = gpd.overlay(eez_12nm, inland_waters, how='difference',keep_geom_type=True) # IFA = (territorial seas) + (contiguous zones) 

    # # inshore fishing area (IFA) 
    # IFA = gpd.overlay(eez_12nm, eez_24nm, how='union',keep_geom_type=True) # IFA = (territorial seas) + (contiguous zones) 
    # IFA_poly=unary_union([eez_12nm.at[0,'geometry'],eez_24nm.at[0,'geometry']]) # shpfile equivalent of IFA
    # alternative based on 12NM inshore exclusive zone (IEZ) 
    IFA = eez_12nm  
    IFA_poly=eez_12nm.at[0,'geometry'] # shpfile equivalent of IFA

    ## parameters
    simulation_type = 'NO CLOSURE' # NO CLOSURE, TOTAL CLOSURE, CENTRAL UPWELLING CLOSURE
    closure_month = ['July'] # July, August, September (if NO CLOSURE, no need to set)
    
    # fish characteristics
    init_fish = 1300 # initial number of fish
    move_fish = 10309 # side length of square cell (meters/day)(diagonal gives the half-maximum-distance covered)
    growth_prob = 0.693 # maximum intrinsic growth rate
    K = 2600 # carrying capacity of fishing ground
    
    # Fisher characteristics
    num_pirogue = 140 # number of pirogues 
    catchability = 0.5 # catchability of pirogues
    frac_coop = 1 # fraction of cooperators (1-frac_coop = frac_noncoop)
    coop_effort = 1 # effort of cooperators (fraction of fish to potentially harvest)
    noncoop_effort = 1 # effort of noncooperators (fraction of fish to potentially harvest)
    low_end = 0 # low end temperature ranging from  0 to 1 (default is 0)
    
    # resource_threshold = 14  # resource threshold (or K/number of cells = 2600/181 =14)
    # Beta = 0.3   # sensitivity for updated effort (0-1)
    
    time = 0 
    time1 = [time]
    month = 'July' # start month must always be July
    num_days = 91 # number of simulation days 
    
    TOTAL_CATCH = [0]
    CURRENT_CATCH =[0]
    TOTAL_FISH = [init_fish]
    AVERAGE_EFFORT = [coop_effort]
     
Param_Initialize() 


### setting square cell on fishing ground

In [ ]:
def Regular_Cell():
    global cell, cells, cells_eez_12nm, cells_eez_24nm
    
    xmin, ymin, xmax, ymax = IFA_poly.bounds # get the bounds of IFA

    #cell size 
    # cell_size = math.sqrt(min_cell_area) # length of square cell
    cell_size = move_fish # side length of square side 
    # create the cells in a loop
    polygons = []
    for x0 in np.arange(xmin, xmax, cell_size):
        for y0 in np.arange(ymin, ymax, cell_size):

            # bounds of polygon
            x1 = x0+cell_size 
            y1 = y0+cell_size

            polygons.append(box(x0, y0, x1, y1))
        
    # form geodataframe 
    cell = gpd.GeoDataFrame(polygons,geometry='geometry',columns=['geometry'],crs=IFA.crs) 
    cells = gpd.overlay(IFA, cell, how='intersection') # only cells within the IFA
    cells['area'] = cells.apply(lambda row: row['geometry'].area, axis=1)
    
    cells_eez_12nm = gpd.overlay(eez_12nm, cell, how='intersection') # only cells within the ees_12nm
    cells_eez_12nm['area'] = cells_eez_12nm.apply(lambda row: row['geometry'].area, axis=1)
    cells_eez_24nm = gpd.overlay(eez_24nm, cell, how='intersection') # only cells within the ees_24nm
    cells_eez_24nm['area'] = cells_eez_24nm.apply(lambda row: row['geometry'].area, axis=1)
    

Regular_Cell() 


## ecologically or biologically significant marine area (EBSA)

In [ ]:
def Hotspots_upwelling():
    global upwelling_gdf_bound, cells_without_upwelling_gdf, normalised_upwelling_temp, scaled_upwelling_temp, low_end
    
    lat_min = 4.6
    lat_max = 5.4
    lon_min = -3.1
    lon_max = -1.7
    # Create a GeoDataFrame for the upwelling zone rectangle
    upwelling_geom = box(lon_min, lat_min, lon_max, lat_max)
    upwelling_gdf = gpd.GeoDataFrame({'Zone': ['Central Upwelling Zone']}, geometry=[upwelling_geom], crs="EPSG:4326") # the crs coordinate with the given. coordinates
    upwelling_gdf  = upwelling_gdf.to_crs(epsg=32630)
    upwelling_gdf_bound = gpd.overlay(cells, upwelling_gdf, how='intersection') # boundary of central upwelling area within the IFA
    upwelling_gdf_bound['area'] = upwelling_gdf_bound.apply(lambda row: row['geometry'].area, axis=1)
    

    mu = 45  # peak upwelling day
    sigma = 15 # std of upwelling day
    day = np.arange(0,91,1) # day
    upwelling = (1 / (sigma * np.sqrt(2 * np.pi)))  * np.exp(-0.5 * ((day - mu) / sigma)**2) # normal dist
    upwelling_temp = -upwelling  # inverted normal dist
    # normalize the upwelling_temp values to range [low_end, 1]
    normalised_upwelling_temp = low_end + ((upwelling_temp - np.min(upwelling_temp)) * (1-low_end)) / (np.max(upwelling_temp) - np.min(upwelling_temp))
    #scale the normalised_upwelling_temp values to range [21, 26]
    scaled_upwelling_temp = normalised_upwelling_temp * (26 - 21) + 21
    
    # assign temperatures based on zones (hotspot, central upwelling, none)
    cells['temperature'] = cells.apply(lambda row: (scaled_upwelling_temp[time]-random.uniform(0,0.5)) if any(row['geometry'].within(row1) or row['geometry'].intersects(row1) or row['geometry'].touches(row1) for row1 in upwelling_gdf_bound['geometry']) # hotspots temp
                                       else (scaled_upwelling_temp[time]+random.uniform(0,0.5)), 
                                       axis=1)
    
    # upwelling_gdf_without_hotspot_gdf = gpd.overlay(upwelling_gdf_bound, hotspot_gdf, how='difference') # boundary of central upwelling area within the IFA without hotspots
    cells_without_upwelling_gdf = gpd.overlay(cells, upwelling_gdf, how='difference') # boundary of central upwelling area within the IFA
  
Hotspots_upwelling()


## fish task

### initialise fish 

In [ ]:
def Fish_Initialize() :
    global fish_geodata
    xmin, ymin, xmax, ymax = IFA_poly.bounds # get the bounds of IFA
    fish_geodata = gpd.GeoDataFrame(geometry='geometry', columns=['x','y', 'agent type', 'geometry'],crs=IFA.crs) # empty geodataframe

    for fish in range(init_fish): # create fishes 
        while True:
            fish_x = random.uniform(xmin, xmax) # uniformly randomly distributed
            fish_y = random.uniform(ymin, ymax)
            pnt = Point(fish_x, fish_y)
            # if IFA_poly.contains(pnt):
            if cells['geometry'].contains(pnt).any():
                fish_geodata.at[fish,'geometry'] = pnt
                fish_geodata.at[fish,'x'] = fish_x
                fish_geodata.at[fish,'y'] = fish_y
                fish_geodata.at[fish,'agent type'] = 'fish'
                break

Fish_Initialize()  


### fish updating (movements and reproduction)

In [ ]:
def Fish_Update() :
    global fish_geodata, normalised_upwelling_temp

    if len(fish_geodata) > 0:
        # randomly sample a geometry from the geodataframe
        focal_fish_index=random.randrange(len(fish_geodata)) # index of focal fish
        focal_fish=fish_geodata.at[focal_fish_index,'geometry'] # geometry of focal fish
        focal_fish_poly=cells.loc[cells['geometry'].contains(focal_fish)] # cell containing focal fish 

        # Moore neighborhood cells of focal fish cell
        cells['touch']=cells.apply(lambda row: row['geometry'].touches(focal_fish_poly.at[focal_fish_poly.index[0],'geometry']), axis=1) 
        focal_fish_poly_neighbors=cells.loc[cells['touch'] == True]

        if len(focal_fish_poly_neighbors) > 0:
            # the neighborhood cell with the minimum temperature
            focal_fish_poly_neighbors_minimum = focal_fish_poly_neighbors.loc[focal_fish_poly_neighbors['temperature'] == focal_fish_poly_neighbors['temperature'].min() ]

            # check whether its temperature is greater than focal fish cell temp (else stay)
            if focal_fish_poly['temperature'].min() > focal_fish_poly_neighbors_minimum['temperature'].min() :
                xmin, ymin, xmax, ymax = focal_fish_poly_neighbors_minimum.at[focal_fish_poly_neighbors_minimum.index[0],'geometry'].bounds # get the bounds 
                
                ## after 100 steps and no random points within target cell found, use the centroid
                steps = 0
                found = False
                while steps <= 100:
                    steps +=1
                    x_coord = random.uniform(xmin, xmax) # uniformly randomly distributed
                    y_coord = random.uniform(ymin, ymax)
                    pnt = Point(x_coord, y_coord)
                    if focal_fish_poly_neighbors_minimum.at[focal_fish_poly_neighbors_minimum.index[0],'geometry'].contains(pnt):
                        found = True
                        fish_geodata.at[focal_fish_index,'x'] =  x_coord 
                        fish_geodata.at[focal_fish_index,'y'] =  y_coord
                        fish_geodata.at[focal_fish_index,'geometry'] =  Point(fish_geodata.at[focal_fish_index,'x'], fish_geodata.at[focal_fish_index,'y']) 
                        break
                        
                if not found:  
                    center= (focal_fish_poly_neighbors_minimum.at[focal_fish_poly_neighbors_minimum.index[0],'geometry']).centroid # set centroid as point
                    fish_geodata.at[focal_fish_index,'x'] =  center.x 
                    fish_geodata.at[focal_fish_index,'y'] =  center.y
                    fish_geodata.at[focal_fish_index,'geometry'] =  Point(fish_geodata.at[focal_fish_index,'x'], fish_geodata.at[focal_fish_index,'y']) 


    
    # simulate natural mortality 
    natural_mortality = 0.00183
    if random.random() <  natural_mortality: 
        fish_geodata.drop(focal_fish_index, inplace=True) # remove the fish
        fish_geodata.reset_index(drop=True, inplace=True)  # reset the index of geodataframe

    else:
        # simulate reproduction 
        max_repro = growth_prob * K / 4  # maximum at N = K/2
        logistic_prob = (growth_prob * len(fish_geodata) * (1 - len(fish_geodata) / K)) / max_repro
        combined_prob = logistic_prob * (1-normalised_upwelling_temp[time])
        
        # logistic reproduction
        if random.random() <  combined_prob: 
            fish_geodata.loc[len(fish_geodata)] = [focal_fish.x , focal_fish.y, 'fish', Point(focal_fish.x , focal_fish.y) ]

   
Fish_Update()


### pirogue tasks

#### spatial query function to find points within polygons

In [ ]:
def intersect_using_spatial_index(source_gdf, intersecting_gdf):
    """
    Conduct spatial intersection using spatial index for candidates GeoDataFrame to make queries faster.
    Note, with this function, you can have multiple Polygons in the 'intersecting_gdf' and it will return all the points 
    intersect with ANY of those geometries.
    """
    source_sindex = source_gdf.sindex
    possible_matches_index = []
    
    # 'itertuples()' function is a faster version of 'iterrows()'
    for other in intersecting_gdf.itertuples():
        bounds = other.geometry.bounds
        c = list(source_sindex.intersection(bounds))
        possible_matches_index += c
    
    # Get unique candidates
    unique_candidate_matches = list(set(possible_matches_index))
    possible_matches = source_gdf.iloc[unique_candidate_matches]

    # Conduct the actual intersect
    # result = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]
    result = possible_matches.loc[possible_matches.intersects(intersecting_gdf.union_all())]
    
    return result


#### Initialize pirogues

In [ ]:
def Pirogue_Initialize() :
    global pirogue_geodata, simulation_type, cells_pirogue_movement_all, cells_pirogue_movement_hotspots, cells_pirogue_movement_upwelling

    pirogue_geodata = gpd.GeoDataFrame(geometry='geometry',columns=['x','y', 'agent type', 'geometry', 'catch', 'current catch', 'effort', 'currentcatch_time0' ],crs=IFA.crs) # empty geodataframe
    
    
    if simulation_type in ['NO CLOSURE', 'TOTAL CLOSURE']:
        cells_pirogue_movement_all = cells 
        # cells_pirogue_movement_all = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry', crs=IFA.crs)
        
    ## CENTRAL UPWELLING CLOSURE by month
    if simulation_type == 'CENTRAL UPWELLING CLOSURE' and month in closure_month and month in ['July', 'August', 'September']:
    # if simulation_type == 'CENTRAL UPWELLING CLOSURE' and closure_month == month and month in ['July', 'August', 'September']:
        cells_pirogue_movement_all = gpd.overlay(cells, upwelling_gdf_bound, how='difference') 
    else:
        cells_pirogue_movement_all = cells
      
    
     # setting the characteristics of pirogues
    for pirogue in range(num_pirogue): 
        while True: 
            # cell_pirogue_index=random.randrange(len(cells_pirogue_movement_all)) # random index 
            cell_pirogue_index=random.choice(cells_pirogue_movement_all.index.tolist()) # random index
            cell_pirogue=cells_pirogue_movement_all.at[cell_pirogue_index,'geometry']  # geometry of cell to contain pirogues
            xmin, ymin, xmax, ymax = cell_pirogue.bounds # get the bounds of the cell to contain pirogue
            pirogue_x = random.uniform(xmin, xmax)
            pirogue_y = random.uniform(ymin, ymax)
            pnt = Point(pirogue_x,pirogue_y)

            if cell_pirogue.contains(pnt):
                pirogue_geodata.at[pirogue,'geometry'] = pnt
                pirogue_geodata.at[pirogue,'x'] = pirogue_x
                pirogue_geodata.at[pirogue,'y'] = pirogue_y
                pirogue_geodata.at[pirogue,'agent type'] = 'pirogue'
                pirogue_geodata.at[pirogue,'catch'] = 0
                pirogue_geodata.at[pirogue,'current catch'] = 0
                pirogue_geodata.at[pirogue,'effort'] = coop_effort if  pirogue <  int(frac_coop * num_pirogue) else noncoop_effort          # set their cooperative-trait
                pirogue_geodata.at[pirogue,'currentcatch_time0'] = 0
                break

Pirogue_Initialize()


#### pirogue updating (movements and harvest)

In [ ]:
def Pirogue_Update() :
    global pirogue_geodata, focal_pirogue, focal_pirogue_cell, focal_pirogue_neighbors, neighbors_without_focal_pirogue
    global moore_neighbor_cells, focal_pirogue_cell_fishes, simulation_type, cells_pirogue_movement_all, Beta, resource_threshold
    # global simulation_type, cells_pirogue_movement_all,cells_pirogue_movement_hotspots, cells_pirogue_movement_upwelling
    
    #randomly sample a pirogue
    focal_pirogue_index=random.randrange(len(pirogue_geodata)) # index of focal pirogue
    focal_pirogue=pirogue_geodata.loc[focal_pirogue_index:focal_pirogue_index]  # sample focal piorgue using the index

    ## cell containing the focal_pirogue
    focal_pirogue_cell = cells_pirogue_movement_all.loc[cells_pirogue_movement_all['geometry'].contains(focal_pirogue.at[focal_pirogue_index,'geometry'])]
    focal_pirogue_neighbors=intersect_using_spatial_index(source_gdf=pirogue_geodata, intersecting_gdf=focal_pirogue_cell) # find all pirogues in the focal pirogue cell
    neighbors_without_focal_pirogue = focal_pirogue_neighbors.loc[focal_pirogue_neighbors['geometry'] != focal_pirogue.at[focal_pirogue_index,'geometry'] ] # neighbor pirogues without focal pirogue
    
    # Moore neighborhood cells of focal pirogue cell
    cells_pirogue_movement_all['touch1']=cells_pirogue_movement_all.apply(lambda row: row['geometry'].intersects(focal_pirogue_cell['geometry']), axis=1) 
    focal_pirogue_cell_neighbors=cells_pirogue_movement_all.loc[cells_pirogue_movement_all['touch1'] == True] # all moore cells including focal cell
    
    # if len(focal_pirogue_cell_neighbors) > 0:
    #     moore_neighbors=intersect_using_spatial_index(source_gdf=pirogue_geodata, intersecting_gdf=focal_pirogue_cell_neighbors) # all pirogues in moore cells (including focal cell)
    #     moore_neighbors_without_focal_pirogue = moore_neighbors.loc[moore_neighbors['geometry'] != focal_pirogue.at[focal_pirogue_index,'geometry'] ] 
    #     if len(moore_neighbors_without_focal_pirogue) > 0: 
    #         moore_maxcatch_idx=moore_neighbors_without_focal_pirogue['current catch'].idxmax()  
    #         moore_maxcatch_geom = moore_neighbors_without_focal_pirogue.loc[moore_maxcatch_idx, 'geometry']
    #         moore_maxcatch_cell=cells_pirogue_movement_all.loc[cells_pirogue_movement_all['geometry'].contains(moore_maxcatch_geom)] 
    #         if ((pirogue_geodata.at[focal_pirogue_index,'current catch']) <= (pirogue_geodata.at[moore_maxcatch_idx,'current catch'])):
    #             Xmin, Ymin, Xmax, Ymax = moore_maxcatch_cell.at[moore_maxcatch_cell.index[0],'geometry'].bounds 
    #             steps = 0
    #             found = False        
    #             while steps <= 100:
    #                 steps +=1
    #                 x_coord = random.uniform(Xmin, Xmax) # uniformly randomly distributed
                #     y_coord = random.uniform(Ymin, Ymax)
                #     pnt = Point(x_coord, y_coord)
                #     if moore_maxcatch_cell.at[moore_maxcatch_cell.index[0],'geometry'].contains(pnt): # if max-catch neighbor is greater than focal pirogue then stay in same cell
                #         found = True
                #         pirogue_geodata.at[focal_pirogue_index,'x'] =  x_coord 
                #         pirogue_geodata.at[focal_pirogue_index,'y'] =  y_coord
                #         pirogue_geodata.at[focal_pirogue_index,'geometry'] =  Point(pirogue_geodata.at[focal_pirogue_index,'x'], pirogue_geodata.at[focal_pirogue_index,'y']) 
                #         break

                # if not found:  
                #     center= (moore_maxcatch_cell.at[moore_maxcatch_cell.index[0],'geometry']).centroid # set centroid as point
                #     pirogue_geodata.at[focal_pirogue_index,'x'] =  center.x 
                #     pirogue_geodata.at[focal_pirogue_index,'y'] =  center.y
                #     pirogue_geodata.at[focal_pirogue_index,'geometry'] =  Point(pirogue_geodata.at[focal_pirogue_index,'x'], pirogue_geodata.at[focal_pirogue_index,'y']) 

     
    
    if len(neighbors_without_focal_pirogue) > 0: # if neighbors in focal cell
        idx_maxcatch_neighbor=neighbors_without_focal_pirogue['current catch'].idxmax() # index of neighbor with max current catch within current cell
        if ((pirogue_geodata.at[idx_maxcatch_neighbor,'current catch']) <= (pirogue_geodata.at[focal_pirogue_index,'current catch'])): # if focal pirogue catch is greater than max-catch neighbor (in same cell)
            # Xmin, Ymin, Xmax, Ymax = focal_pirogue_cell.at[focal_pirogue_cell.index[0],'geometry'].bounds 
            if len(focal_pirogue_cell_neighbors) > 0: # if moore neighbor cells exist
                idx_focal_pirogue_cell_neighbors = random.choice(focal_pirogue_cell_neighbors.index.tolist()) # random index of a moore neighbor cell
                focal_pirogue_cell = focal_pirogue_cell_neighbors.loc[idx_focal_pirogue_cell_neighbors : idx_focal_pirogue_cell_neighbors]
                Xmin, Ymin, Xmax, Ymax = focal_pirogue_cell_neighbors.at[idx_focal_pirogue_cell_neighbors,'geometry'].bounds 
        
                steps = 0
                found = False        
                while steps <= 100:
                    steps +=1
                    x_coord = random.uniform(Xmin, Xmax) # uniformly randomly distributed
                    y_coord = random.uniform(Ymin, Ymax)
                    pnt = Point(x_coord, y_coord)
                    if focal_pirogue_cell_neighbors.at[idx_focal_pirogue_cell_neighbors,'geometry'].contains(pnt): # if max-catch neighbor is greater than focal pirogue then stay in same cell
                        found = True
                        pirogue_geodata.at[focal_pirogue_index,'x'] =  x_coord 
                        pirogue_geodata.at[focal_pirogue_index,'y'] =  y_coord
                        pirogue_geodata.at[focal_pirogue_index,'geometry'] =  Point(pirogue_geodata.at[focal_pirogue_index,'x'], pirogue_geodata.at[focal_pirogue_index,'y']) 
                        break

                if not found:  
                    center= (focal_pirogue_cell_neighbors.at[idx_focal_pirogue_cell_neighbors,'geometry']).centroid # set centroid as point
                    pirogue_geodata.at[focal_pirogue_index,'x'] =  center.x 
                    pirogue_geodata.at[focal_pirogue_index,'y'] =  center.y
                    pirogue_geodata.at[focal_pirogue_index,'geometry'] =  Point(pirogue_geodata.at[focal_pirogue_index,'x'], pirogue_geodata.at[focal_pirogue_index,'y']) 

    else: # no neighbors within cells
        if len(focal_pirogue_cell_neighbors) > 0: # if moore neighbor cells exist
            idx_focal_pirogue_cell_neighbors = random.choice(focal_pirogue_cell_neighbors.index.tolist()) # random index of a moore neighbor cell
            focal_pirogue_cell = focal_pirogue_cell_neighbors.loc[idx_focal_pirogue_cell_neighbors : idx_focal_pirogue_cell_neighbors]
            Xmin, Ymin, Xmax, Ymax = focal_pirogue_cell_neighbors.at[idx_focal_pirogue_cell_neighbors,'geometry'].bounds 

            steps = 0
            found = False        
            while steps <= 100:
                steps +=1
                x_coord = random.uniform(Xmin, Xmax) # uniformly randomly distributed
                y_coord = random.uniform(Ymin, Ymax)
                pnt = Point(x_coord, y_coord)
                if focal_pirogue_cell_neighbors.at[idx_focal_pirogue_cell_neighbors,'geometry'].contains(pnt): # if max-catch neighbor is greater than focal pirogue then stay in same cell
                    found = True
                    pirogue_geodata.at[focal_pirogue_index,'x'] =  x_coord 
                    pirogue_geodata.at[focal_pirogue_index,'y'] =  y_coord
                    pirogue_geodata.at[focal_pirogue_index,'geometry'] =  Point(pirogue_geodata.at[focal_pirogue_index,'x'], pirogue_geodata.at[focal_pirogue_index,'y']) 
                    break

            if not found:  
                center= (focal_pirogue_cell_neighbors.at[idx_focal_pirogue_cell_neighbors,'geometry']).centroid # set centroid as point
                pirogue_geodata.at[focal_pirogue_index,'x'] =  center.x 
                pirogue_geodata.at[focal_pirogue_index,'y'] =  center.y
                pirogue_geodata.at[focal_pirogue_index,'geometry'] =  Point(pirogue_geodata.at[focal_pirogue_index,'x'], pirogue_geodata.at[focal_pirogue_index,'y']) 


        
    # harvest fish from the focal pirogue cell for constant effort
    focal_pirogue_cell_fishes=intersect_using_spatial_index(source_gdf=fish_geodata, intersecting_gdf=focal_pirogue_cell ) # fishes in focal pirogue cell 
    focal_pirogue_catch = int(catchability * pirogue_geodata.at[focal_pirogue_index,'effort'] * len(focal_pirogue_cell_fishes)) # number of fishes to harvest
    pirogue_geodata.at[focal_pirogue_index,'current catch'] += focal_pirogue_catch # add harvest as current catch 
    pirogue_geodata.at[focal_pirogue_index,'catch'] += focal_pirogue_catch  # add harvest to total catch 
    fish_index_remove=list(focal_pirogue_cell_fishes.sample(n=focal_pirogue_catch).index.values) # index of fishes to remove
    fish_geodata.drop(fish_index_remove, inplace=True) # remove the fish
    fish_geodata.reset_index(drop=True, inplace=True)  # reset the index of geodataframe


    # # Functions from the model for variable effort
    # def resource_based_target(cell_fishes, resource_threshold, Beta):
    #     return 1 / (1 + np.exp(-Beta * (cell_fishes  - resource_threshold)))
    # def update_effort(cell_fishes, resource_threshold, Beta):
    #     new_effort = resource_based_target(cell_fishes, resource_threshold, Beta)
    #     return np.clip(new_effort, 0, 1)

    # # Fishes in focal pirogue cell
    # focal_pirogue_cell_fishes = intersect_using_spatial_index(source_gdf=fish_geodata,intersecting_gdf=focal_pirogue_cell)
    # cell_fishes = len(focal_pirogue_cell_fishes)
    
    # # Compute updated effort
    # updated_effort=update_effort(cell_fishes, resource_threshold, Beta)
    # pirogue_geodata.at[focal_pirogue_index,'effort'] = float(updated_effort)

    # # harvest fishes from focal pirogue cell
    # # focal_pirogue_cell_fishes=intersect_using_spatial_index(source_gdf=fish_geodata, intersecting_gdf=focal_pirogue_cell ) # fishes in focal pirogue cell 
    # focal_pirogue_catch = int(catchability * pirogue_geodata.at[focal_pirogue_index,'effort'] * cell_fishes) # number of fishes to harvest
    # pirogue_geodata.at[focal_pirogue_index,'current catch'] += focal_pirogue_catch # add harvest as current catch 
    # pirogue_geodata.at[focal_pirogue_index,'catch'] += focal_pirogue_catch  # add harvest to total catch 
    # fish_index_remove=list(focal_pirogue_cell_fishes.sample(n=focal_pirogue_catch).index.values) # index of fishes to remove
    # fish_geodata.drop(fish_index_remove, inplace=True) # remove the fish
    # fish_geodata.reset_index(drop=True, inplace=True)  # reset the index of geodataframe
    
    
Pirogue_Update() 


### observing fishing ground

In [ ]:
def Observe():
    ### invoking spatial characters for plotting
    eez_12nm_ivorycoast  = gpd.read_file(os.path.join(os.getcwd(), 'eez_12nm_ivorycoast/eez_12nm.shp')) # Exclusive Economic Zone (200 nautical miles)
    eez_12nm_ivorycoast= eez_12nm_ivorycoast.to_crs(epsg=32630) # reproject crs

    #Ghana map
    place_name = 'Ghana'
    boundaries  = ox.geocode_to_gdf(place_name)
    boundaries = boundaries.to_crs(IFA.crs)
    #Ivory coast map
    place_name1 = 'Ivory coast'
    boundaries1  = ox.geocode_to_gdf(place_name1)
    boundaries1 = boundaries1.to_crs(IFA.crs)
    #land
    upper_land = gpd.overlay(cell, boundaries, how='intersection') # upper part of land based on map
    upper_land1 = gpd.overlay(cells_eez_12nm, upper_land, how='symmetric_difference') # upper part of land not including eez_12nm  
    #ocean
    # lower_sea = gpd.overlay(cells_eez_24nm, upper_land, how='union',keep_geom_type=True) # lower sea of Ghana
    lower_sea = gpd.overlay(cells_eez_12nm, upper_land, how='union',keep_geom_type=True) # lower sea of Ghana
    lower_sea1 = gpd.overlay(cell, lower_sea, how='symmetric_difference')
    
    left_side = gpd.overlay(boundaries1, lower_sea1, how='intersection') # side ivory coast 
    left_side1 = gpd.overlay(lower_sea1, left_side, how='symmetric_difference') # atlantic ocean 
    left_side3 = gpd.overlay(eez_12nm_ivorycoast, left_side , how='intersection')

    ### Plotting
    fig, ax = plt.subplots(figsize=(16,8))
    minx1, miny1, maxx1, maxy1 = IFA['geometry'].total_bounds# IFA bounds
     
    # create a legend: we'll plot empty lists with the desired color, label, symbol
    for facecol, label, edgecol, symb, alph in [
                                              # ('white','Spawning hotspots','red','s', 1),
                                              ('white','EBSA','mediumblue','s', 1),
                                              # ('white','Non-EBSA','black','s', 1), 
                                              # ('red','Closed to Fishing ','red','s', 0.3),
                                              ('black','Fish Agent','black','.', 1) ,
                                              ('white','Fisher Agent','black','o', 1) 
                                              ]:
        ax.scatter([], [], facecolor=facecol, s=100, label=label, alpha=alph, edgecolors=edgecol, marker=symb,linewidths=1.5 )
        ax.legend(facecolor="white", edgecolor="none", prop={"size":12}, loc=(0.5,0.02),ncol=1 ) #loc=(0.7,0.02)
   
    # cells.plot(ax=ax,facecolor="none", edgecolor='black', lw=1, zorder=1, alpha=0.5)
    vmin = 21 #cells['temperature'].min()  
    vmax = 26 #cells['temperature'].max()  
    tick_values = np.linspace(vmin, vmax, num=5)
    cells.plot(ax=ax, column='temperature', cmap='turbo', legend=True, edgecolor='black', lw=1, alpha=0.5, vmin=vmin, vmax=vmax,
               legend_kwds={'shrink': 0.5, 'aspect': 20, 'orientation': 'vertical', 'label': 'Temperature (°C)', 'ticks': tick_values})
        
    upwelling_gdf_bound.plot(ax=ax,facecolor="none", edgecolor='blue', lw=1.6, zorder=2, alpha=0.5)

    fish_geodata.plot(ax=ax,color='black', edgecolor='black' ,marker = '.', markersize=2, zorder=2)

    if simulation_type in ['NO CLOSURE', 'CENTRAL UPWELLING CLOSURE']:
        pirogue_geodata.plot(ax=ax,facecolor='none', edgecolor='black' ,marker = 'o', linewidth =0.8,markersize=5,zorder=2)
    
    # if simulation_type == 'TOTAL CLOSURE' and closure_month != month and month in ['July', 'August', 'September']:
    if simulation_type == 'TOTAL CLOSURE' and month not in closure_month and month in ['July', 'August', 'September']:
        pirogue_geodata.plot(ax=ax,facecolor='none', edgecolor='black' ,marker = 'o', linewidth =0.8,markersize=5,zorder=2)
  
    
    upper_land1.plot(ax=ax,facecolor='grey', edgecolor='none', lw=1,zorder=1,alpha=0.5)
    left_side1.plot(ax=ax,facecolor='skyblue', edgecolor='none', lw=1,zorder=1,alpha=0.5)
    left_side3.plot(ax=ax,facecolor='skyblue', edgecolor='none', lw=1,zorder=1,alpha=0.5)    
    
    ax.set_xlabel('Longitude  (km)',fontsize=15)
    ax.set_ylabel('Latitude  (km)',fontsize=15)
    

    if time > 0:
        if simulation_type in ['TOTAL CLOSURE', 'CENTRAL UPWELLING CLOSURE']:
            ax.set_title(str(simulation_type) + ':' + ' ' + 'closure month =' + ' ' + f"[{', '.join(map(str, closure_month))}]" + ',' + ' ' + 'day =' + ' ' + str(int((time-1)%30 + 1 )) + ',' + ' ' + 'month =' + ' ' + month, fontsize=15) # title
        elif simulation_type == 'NO CLOSURE':
            ax.set_title(str(simulation_type) + ':' + ' ' + 'non-closure month =' + ' ' + f"[{', '.join(map(str, ['July', 'August', 'September']))}]" + ',' + ' ' + 'day =' + ' ' + str(int((time-1)%30 + 1)) + ',' + ' ' + 'month =' + ' ' + month, fontsize=15) # title
    else:
        if simulation_type in ['TOTAL CLOSURE', 'CENTRAL UPWELLING CLOSURE']:
            ax.set_title(str(simulation_type) + ':' + ' ' + 'closure month =' + ' ' + f"[{', '.join(map(str, closure_month))}]" + ',' + ' ' + 'day =' + ' ' + str(int(time)) + ',' + ' ' + 'month =' + ' ' + month, fontsize=15) # title
        elif simulation_type == 'NO CLOSURE':
            ax.set_title(str(simulation_type) + ':' + ' ' + 'non-closure month =' + ' ' + f"[{', '.join(map(str, ['July', 'August', 'September']))}]" + ',' + ' ' + 'day =' + ' ' + str(int(time)) + ',' + ' ' + 'month =' + ' ' + month, fontsize=15) # title

 
    ax.text(850 *1000, 650 * 1000, 'Ghana ',fontsize=16)
    # ax.text(850 *1000, 550 * 1000, 'Atlantic Ocean ',fontsize=16)
    ax.text(850 *1000, 590 * 1000, 'Atlantic Ocean ',fontsize=16)
    
    # set axis limit as boundaries of Ghana
    ax.set_xlim(([minx1, maxx1]))
    ax.set_ylim(([miny1, maxy1])) 
    # generate evenly spaced ticks including endpoints
    xticks = np.linspace(minx1,maxx1, num=6)  # 6 ticks from 0 to 10
    yticks = np.linspace(miny1,maxy1, num=6)  # 7 ticks from 20 to
    # set custom ticks
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
  # # ax.set_yticks([]) ; ax.set_xticks([]) 
    # ax.set_axis_off()

    
    # make axis number label not scientific notation
    for axis in [ax.xaxis, ax.yaxis]:
        formatter = ScalarFormatter()
        formatter.set_scientific(False)
        axis.set_major_formatter(formatter)
           
    # # change axis from meters to kilometers
    m2km = lambda x, _: f'{x/1000:g}'
    ax.xaxis.set_major_formatter(m2km)
    ax.yaxis.set_major_formatter(m2km)
    
    # make insert figure within the main plot
    axins = inset_axes(ax,width='40%', height='40%', bbox_to_anchor=(0.07, 0.3, 0.5, 0.7), bbox_transform=ax.transAxes, loc='upper left') # tuple is (x0, y0, width, height)
    axins1 = inset_axes(ax,width='40%', height='40%', bbox_to_anchor=(0.37, 0.3, 0.5, 0.7), bbox_transform=ax.transAxes, loc='upper left') 
    axins2 = inset_axes(ax,width='40%', height='40%', bbox_to_anchor=(0.49, 0.12, 0.5, 0.7), bbox_transform=ax.transAxes, loc='lower right') 
    axins.plot(time1,TOTAL_FISH, color='black',linewidth=2)
    axins1.plot(time1,CURRENT_CATCH, color='black',linewidth=2)
    axins2.bar(time1, AVERAGE_EFFORT, color='white', edgecolor='black', width=0.8)
    axins.set_xlabel('Time (days)',fontsize=12)
    axins.set_ylabel('Fish ($10^3$)',fontsize=12,color='black')
    axins1.set_xlabel('Time (days)',fontsize=12,color='black')
    axins1.set_ylabel('Catch ($10^3$)',fontsize=12,color='black')
    axins2.set_xlabel('Time (days)',fontsize=12,color='black')
    axins2.set_ylabel('Avg. Effort',fontsize=12,color='black')
    inset_axis = lambda x, _: f'{x/1000:g}' # setting in 10^3
    axins.yaxis.set_major_formatter(inset_axis)
    axins1.yaxis.set_major_formatter(inset_axis)
     
    # plt.tight_layout()
    # plt.subplots_adjust(top=0.85) 
    plt.savefig('day_%04d.png' %time, bbox_inches='tight', pad_inches=0.1, dpi=1000) 


# plt.clf
Observe()
# plt.show() 


## reset geometry of pirogues during to closure_month

In [ ]:
def reset_geometry():
    while True: 
        cell_pirogue_index=random.choice(cells_pirogue_movement_all.index.tolist()) # random index
        cell_pirogue=cells_pirogue_movement_all.at[cell_pirogue_index,'geometry']  # geometry of cell to contain pirogues
        xmin, ymin, xmax, ymax = cell_pirogue.bounds # get the bounds of the cell to contain pirogue
        pirogue_x = random.uniform(xmin, xmax)
        pirogue_y = random.uniform(ymin, ymax)
        pnt = Point(pirogue_x,pirogue_y)
        if cell_pirogue.contains(pnt):
            return pnt   
            

### updating fish and pirogues asyncronously

In [ ]:
def update_one_time():
    global time, time1, month, closure_month, cells_pirogue_movement_all
    time += 1  # update time
    
    # update fish
    if len(fish_geodata) > 0:
        t = 0.
        while t < 1 :
            t += 1. / len(fish_geodata) 
            Fish_Update()
    
    # update pirogues
    if simulation_type in ['NO CLOSURE', 'CENTRAL UPWELLING CLOSURE']:
        t = 0.
        while t < 1 :
            t += 1. / len(pirogue_geodata) 
            # t += 1. / (len(pirogue_geodata)/3) # takes 3 days for all to update (meaning on average pirogues updates once every three days)
            Pirogue_Update() 
    elif simulation_type == 'TOTAL CLOSURE' and month not in closure_month and month in ['July', 'August', 'September']:
        t = 0.
        while t < 1 :
            t += 1. / len(pirogue_geodata) 
            # t += 1. / (len(pirogue_geodata)/3) # takes 3 days for all to update (meaning on average pirogues updates once every three days)
            Pirogue_Update()


    # assign temperatures based on zones (hotspot, central upwelling, none)
    cells['temperature'] = cells.apply(lambda row: (scaled_upwelling_temp[time]-random.uniform(0,0.5)) if any(row['geometry'].within(row1) or row['geometry'].intersects(row1) or row['geometry'].touches(row1) for row1 in upwelling_gdf_bound['geometry']) # hotspots temp
                                       else (scaled_upwelling_temp[time]+random.uniform(0,0.5)), 
                                       axis=1)
    # set month by time
    month = 'July' if time <= 30 else  'August' if (30 < time <= 60) else 'September'


    ## CENTRAL UPWELLING CLOSURE by month
    if simulation_type == 'CENTRAL UPWELLING CLOSURE' and month in closure_month and month in ['July', 'August', 'September']:
        if time % 30 == 1:
            cells_pirogue_movement_all = gpd.overlay(cells, upwelling_gdf_bound, how='difference')
            pirogue_geodata['geometry'] = pirogue_geodata.apply(lambda row: reset_geometry(), axis=1)
    elif simulation_type == 'CENTRAL UPWELLING CLOSURE' and month not in closure_month and month in ['July', 'August', 'September']:
        if time % 30 == 1:
            cells_pirogue_movement_all = cells
            pirogue_geodata['geometry'] = pirogue_geodata.apply(lambda row: reset_geometry(), axis=1)

       
    # housekeeping the data
    time1.append(time) # update time
    TOTAL_FISH.append(len(fish_geodata)) # update total fishes
    TOTAL_CATCH.append(pirogue_geodata['catch'].sum()) # update total catch
    CURRENT_CATCH.append(pirogue_geodata['current catch'].sum()) # update total current catch
    
    # if simulation_type == 'TOTAL CLOSURE' and month in closure_month and month in ['July', 'August', 'September']:
    #    pirogue_geodata['effort'] = float(0)
    AVERAGE_EFFORT.append(pirogue_geodata['effort'].mean()) # update average effort
    # pirogue_geodata['currentcatch_time%d'% (time)] = pirogue_geodata.apply(lambda row: row['current catch'] , axis =1) # keep current catch at each time step
    # pirogue_geodata['effort_time%d'% (time)] = pirogue_geodata.apply(lambda row: row['effort'] , axis =1) # keep current effort at each time step
    pirogue_geodata[f'currentcatch_time{time}'] = pirogue_geodata['current catch']
    pirogue_geodata[f'effort_time{time}'] = pirogue_geodata['effort']
    pirogue_geodata.to_csv("final_pirogue_geodata.csv", header=True) # convert pirogue geodata to csv (containing currents catches per time) 
    pirogue_geodata['current catch'] = pirogue_geodata.apply(lambda row: 0, axis =1) # reset current catch to zero
   
    csvfile = "updated_simulation_data.csv"   # a csv-file output 
    header = ['time','number_fish','total_catch','current catch', 'average_effort']
    main_data = [time1, TOTAL_FISH, TOTAL_CATCH, CURRENT_CATCH, AVERAGE_EFFORT]
    with open(csvfile, "w") as output:
        writer = csv.writer(output) 
        writer.writerow(header)
        writer.writerows(zip(*main_data))  

# update_one_time() 


### simulate over a number of time steps

In [ ]:
Param_Initialize() # read files and initialise all parameters
Regular_Cell() # set IFA into regular cells
Hotspots_upwelling() # set the hotspots and upwelling
Fish_Initialize() # initialise clusters of fish
Pirogue_Initialize() # initialize the pirogues
# Observe() # plot to observe

for j in range(1, num_days):  # num_days
    update_one_time()
    # Observe()
    
# os.system("ffmpeg -v quiet -r 5 -i day_%04d.png -vcodec mpeg4  -y -s:v 1920x1080 updated_simulation_movie.mp4") # convert png files to a movie


## EXCESS CODE

In [ ]:
# mu = 45  # peak upwelling day
# sigma = 15 # std of upwelling day
# day = np.arange(0,91,1) # day
# upwelling = (1 / (sigma * np.sqrt(2 * np.pi)))  * np.exp(-0.5 * ((day - mu) / sigma)**2) # normal dist
# upwelling_temp = -upwelling  # inverted normal dist
# # normalize the upwelling_temp values to range [0, 1]
# normalised_upwelling_temp = (upwelling_temp - np.min(upwelling_temp)) / (np.max(upwelling_temp) - np.min(upwelling_temp))
# #scale the normalised_upwelling_temp values to range [20, 26]
# scaled_upwelling_temp = normalised_upwelling_temp * (26 - 21) + 21

# plt.clf
# # plt.figure(figsize=(5, 3))
# fig, ax = plt.subplots(figsize=(10,3))
# ax.plot(day, scaled_upwelling_temp)
# ax.set_xlabel('Day',fontsize=12)
# ax.set_ylabel('Temperature (°C)',fontsize=12)
# ax.set_xlim(([0, 90]))
# ax.set_ylim(([20, 28]))
# ax.grid(True)
# plt.savefig('FIGURE', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()



# plt.clf
# # Plotting
# plt.figure(figsize=(10, 6))
# contour = plt.contourf(day_grid, N_grid, combined_prob, levels=50, cmap='viridis')
# plt.colorbar(contour, label='Combined probability of reproduction')
# plt.xlabel('Day')
# plt.ylabel('Population Size')
# plt.title('Combined Effect of logistic growth and upwelling intensity')
# plt.axvline(45, color='red', linestyle='--', label='Peak Upwelling (Day 45)')
# plt.axhline(K/2, color='blue', linestyle='--', label='Peak Logistic Reproduction (K/2)')
# plt.legend()
# plt.tight_layout()
# plt.savefig('reproduction_prob.pdf', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()


In [ ]:
# mu = 45       # peak day of upwelling (run for three months (July-September), 90 days, peak at mid-day of August
# sigma = 15
# time = np.arange(0,91)
# def upwelling_prob(time):
# # Gaussian upwelling season for three months (July-September), 90 days, centred at day 45
#     return np.exp(-0.5 * ((time - mu) / sigma)**2)

# # Logistic reproduction probability normalised
# growth_prob = 0.693
# K=2000
# n = np.linspace(0, K, 2000)
# def logistic_prob(n):
#     max_repro = growth_prob * K / 4  # maximum at N = K/2
#     return (growth_prob * n * (1 - n / K)) / max_repro


# #Create a meshgrid for population and days
# day_grid, N_grid = np.meshgrid(time, n)

# # Compute combined reproduction probability
# combined_prob = logistic_prob(N_grid) * upwelling_prob(day_grid)



# plt.clf
# # Plotting
# # plt.figure(figsize=(6, 4))
# fig, ax = plt.subplots(figsize=(9,5))
# contour = plt.contourf(day_grid, N_grid, combined_prob, levels=50, cmap='viridis')
# plt.colorbar(contour, label='Combined reproduction probability (p)')
# # ax.plot(logistic_prob , color='blue')
# ax.set_xlabel('Day',fontsize=12)
# ax.set_ylabel('Population size',fontsize=12)

# # plt.title('Combined effect of logistic reproduction and upwelling intensity')
# plt.axvline(45, color='red', linestyle='--', label='Peak upwelling (Day 45)')
# plt.axhline(K/2, color='blue', linestyle='--', label='Peak logistic reproduction (K/2)')

# # # set axis limit as boundaries of Ghana
# # ax.set_xlim(([0, 2000]))
# # ax.set_ylim(([0, 1.0]))
# # # plt.title('Combined Effect of logistic growth and upwelling intensity')
# # # plt.axvline(45, color='red', linestyle='--', label='Peak upwelling (Day 45)')
# # plt.axvline(1000, color="red", linestyle="--", label=f"Max at n = {int(1000)}")
# # # plt.axhline(K/2, color='blue', linestyle='--', label='Peak Logistic Reproduction (K/2)')
# plt.legend()
# plt.tight_layout()
# plt.savefig('reproduction_prob.pdf', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()


In [ ]:
# mu = 45  # peak upwelling day
# sigma = 15 # std of upwelling day
# day = np.arange(0,91,1) # day
# upwelling = (1 / (sigma * np.sqrt(2 * np.pi)))  * np.exp(-0.5 * ((day - mu) / sigma)**2) # normal dist
# upwelling_temp = -upwelling  # inverted normal dist
# # normalize the upwelling_temp values to range [0, 1]
# normalised_upwelling_temp = (upwelling_temp - np.min(upwelling_temp)) / (np.max(upwelling_temp) - np.min(upwelling_temp))
# #scale the normalised_upwelling_temp values to range [20, 26]
# scaled_upwelling_temp = normalised_upwelling_temp * (26 - 21) + 21

# plt.clf
# # plt.figure(figsize=(5, 3))
# fig, ax = plt.subplots(figsize=(10,3))
# ax.plot(day, scaled_upwelling_temp)
# ax.set_xlabel('Day',fontsize=12)
# ax.set_ylabel('Temperature (°C)',fontsize=12)
# ax.set_xlim(([0, 90]))
# ax.set_ylim(([20, 28]))
# ax.grid(True)
# plt.savefig('FIGURE', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()



# plt.clf
# # Plotting
# plt.figure(figsize=(10, 6))
# contour = plt.contourf(day_grid, N_grid, combined_prob, levels=50, cmap='viridis')
# plt.colorbar(contour, label='Combined probability of reproduction')
# plt.xlabel('Day')
# plt.ylabel('Population Size')
# plt.title('Combined Effect of logistic growth and upwelling intensity')
# plt.axvline(45, color='red', linestyle='--', label='Peak Upwelling (Day 45)')
# plt.axhline(K/2, color='blue', linestyle='--', label='Peak Logistic Reproduction (K/2)')
# plt.legend()
# plt.tight_layout()
# plt.savefig('reproduction_prob.pdf', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()



In [ ]:
# mu = 45  # peak upwelling day
# sigma = 15 # std of upwelling day
# day = np.arange(0,91,1) # day
# upwelling = (1 / (sigma * np.sqrt(2 * np.pi)))  * np.exp(-0.5 * ((day - mu) / sigma)**2) # normal dist
# upwelling_temp = -upwelling  # inverted normal dist
# # normalize the upwelling_temp values to range [0, 1]
# normalised_upwelling_temp = (upwelling_temp - np.min(upwelling_temp)) / (np.max(upwelling_temp) - np.min(upwelling_temp))
# #scale the normalised_upwelling_temp values to range [20, 26]
# scaled_upwelling_temp = normalised_upwelling_temp * (26 - 21) + 21

# plt.clf
# # plt.figure(figsize=(5, 3))
# fig, ax = plt.subplots(figsize=(10,3))
# ax.plot(day, scaled_upwelling_temp)
# ax.set_xlabel('Day',fontsize=12)
# ax.set_ylabel('Temperature (°C)',fontsize=12)
# ax.set_xlim(([0, 90]))
# ax.set_ylim(([20, 28]))
# ax.grid(True)
# plt.savefig('FIGURE', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()



# plt.clf
# # Plotting
# plt.figure(figsize=(10, 6))
# contour = plt.contourf(day_grid, N_grid, combined_prob, levels=50, cmap='viridis')
# plt.colorbar(contour, label='Combined probability of reproduction')
# plt.xlabel('Day')
# plt.ylabel('Population Size')
# plt.title('Combined Effect of logistic growth and upwelling intensity')
# plt.axvline(45, color='red', linestyle='--', label='Peak Upwelling (Day 45)')
# plt.axhline(K/2, color='blue', linestyle='--', label='Peak Logistic Reproduction (K/2)')
# plt.legend()
# plt.tight_layout()
# plt.savefig('reproduction_prob.pdf', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()



In [ ]:
# # reading shapefiles
# eez  = gpd.read_file("/Users/kwabx/MPA_code/eez/eez.shp") # Exclusive Economic Zone (200 nautical miles)
# eez_12nm  = gpd.read_file("/Users/kwabx/MPA_code/eez_12nm/eez_12nm.shp") # 12 nautical miles zones (territorial seas)
# eez_24nm  = gpd.read_file("/Users/kwabx/MPA_code/eez_24nm/eez_24nm.shp") # 24 nautical miles zones (contiguous zones)

# # reproject crs to metre's
# eez  = eez.to_crs(epsg=25000) 
# eez_12nm  = eez_12nm.to_crs(epsg=25000) 
# eez_24nm  = eez_24nm.to_crs(epsg=25000) 

# # inshore fishing area 
# IFA = gpd.overlay(eez_12nm, eez_24nm, how='union') # IFA = (territorial seas) + (contiguous zones) 
# IFA_poly=unary_union([eez_12nm.at[0,'geometry'],eez_24nm.at[0,'geometry']]) # the multipolygon equivalent of IFA

# eez_area= eez['geometry'].area / 10**6  # compute area in square kilometers
# eez_12nm_area= eez_12nm['geometry'].area / 10**6  
# eez_24nm_area= eez_24nm['geometry'].area / 10**6 
# IFA_area = IFA_poly.area / 10**6


In [ ]:
# Type_MPA = 'single' # type of simulation (nompa, single, spaced, nompa-single, nompa-spaced)
# Frac_MPA = 0.01    # fraction of the fishing ground to set as MPA 
# Dist_MPA = 0.000001     # distance between spaced MPA expressed as a fraction of IFA area
# Time_MPA = 15       # time to terminate nompa and invoke mpa (in days)

# num_clusters = 5 # initialise number of fish clusters
# init_fish = 1000 # initial number of fishes
# point_deviation = 20000 # std deviation from cluster of fish
# rad_repulsion = 1000 # radius of repulsion zone
# rad_orientation = 2000 # radius of orientation zone 
# rad_attraction =  3000 # radius of attraction zone  
# move_fish = 2000 # speed of fish
# growth_prob =  0.3 # maximum intrinsic growth rate
# K = 10000 # carrying capacity of fishing ground

# num_pirogue = 100
# catchability = 0.6

# time = 0 # time for updating
# time1 = [time]
    

In [ ]:
# # # plotting ghana map with IFA and EEZ
# boundaries  = ox.geocode_to_gdf('Ghana')
# boundaries = boundaries.to_crs(eez.crs)

# fig, ax = plt.subplots(figsize=(20,6))
# # fig = plt.figure(figsize=(20, 5))
# # ax = fig.add_subplot(1,1,1)
# boundaries.plot(ax=ax,fc='none', ec='black', alpha=1, zorder=1)
# # eez.plot(ax=ax,fc='none', ec='blue',zorder=1, linewidth=2,alpha=1)
# IFA.plot(ax=ax,fc='green', ec='none',zorder=1,alpha=0.2)
# ax.set_yticks([]) ; ax.set_xticks([]) ; ax.set_axis_off()
# plt.tight_layout()
# plt.savefig('ghana1.pdf', bbox_inches='tight', pad_inches=0.1, dpi=500) 


In [ ]:
# ghana_lake  = gpd.read_file(os.path.join(os.getcwd(), 'ne_50m_lakes/ne_50m_lakes.shp')) # 24 nautical miles zones (contiguous zones)
# ghana_lake  = ghana_lake.to_crs(epsg=32630) 


# # ghana_coast  = gpd.read_file(os.path.join(os.getcwd(), 'ghana_coast/world_countries_coasts.shp')) # 24 nautical miles zones (contiguous zones)
# # ghana_coast  = ghana_coast.to_crs(epsg=32630) 

# ghana_land  = gpd.read_file(os.path.join(os.getcwd(), 'ghana_land/worldcountries_esri_2014.shp')) # 24 nautical miles zones (contiguous zones)
# ghana_land  = ghana_land.to_crs(epsg=32630) 

# # #Ghana map
# # place_name = 'Ghana'
# # boundaries  = ox.geocode_to_gdf(place_name)
# # boundaries = boundaries.to_crs(IFA.crs)

# IFA1 = gpd.overlay(ghana_land,ghana_lake, how='intersection',keep_geom_type=True) # IFA = (territorial seas) + (contiguous zones) 
# # # IFA2 = gpd.overlay(IFA1, boundaries, how='difference',keep_geom_type=True) # IFA = (territorial seas) + (contiguous zones) 

# # # IFA2 = gpd.overlay(ghana_coast, IFA1, how='symmetric_difference',keep_geom_type=True)

# plt.clf
# fig, ax = plt.subplots(figsize=(16,8))
# ghana_land.plot(ax=ax, facecolor='blue', alpha=0.5)
# eez_12nm.plot(ax=ax, facecolor='red', alpha =0.5)
# IFA.plot(ax=ax, facecolor='none', edgecolor= 'black', alpha =0.5)
# boundaries.plot(ax=ax, facecolor='black', alpha=0.5)
# ghana_coast.plot(ax=ax, edgecolor='blue', alpha=0.5)
# ghana_lake.plot(ax=ax, facecolor='blue', alpha=0.5)
# IFA1.plot(ax=ax, facecolor='blue', alpha=0.5)
# plt.show()

In [ ]:
# def MPA_Characteristics():
#     global MPA, Area_MPA, mpa_area, neighbor_cells, MPA_cell_shapefile, MPA_cellx, MPA_ConvexHull, MPA_union
# #--------------------------------------------------------------------------------------------------------    
    
    
#     def mpa_characteristics():
#         global MPA, Area_MPA, mpa_area, neighbor_cells, MPA_cell_shapefile, MPA_cellx
        
#         MPA_start_cell_index=random.randrange(len(cells)) # index of MPA start cell
#         MPA_cell =cells.at[MPA_start_cell_index,'geometry'] # geometry of MPA start cell
#         MPA_cellx = cells.at[MPA_start_cell_index,'geometry'] # just for computing nearest distance to it
#         MPA_cell_shapefile =cells.loc[MPA_start_cell_index : MPA_start_cell_index] # shapefile of MPA start cell
#         # MPA_cell_shapefile = cells.loc[cells['geometry'].contains(IFA_poly.centroid)]
#         # MPA_cell = MPA_cell_shapefile.at[MPA_cell_shapefile.index.values[0],'geometry']

#         # Area_MPA = Frac_MPA * (cells['area'].sum()) # total area of MPA as a fraction of IFA area
#         mpa_area = 0 # initialize MPA area
#         ALL_neighbor_cells = []

#         while True:
#             # print(len(cells))
#             # cells in neighborhood start mpa cell  
#             neighbor_cells = cells.loc[ ((cells['geometry']).distance(MPA_cell) < 5000) & (cells['geometry'] != MPA_cell) \
#                                        # & ((cells.intersection(MPA_cell)).geom_type == "LineString") 
#                                       ]

#             neighbor_cells =  gpd.GeoDataFrame(pd.concat([MPA_cell_shapefile,neighbor_cells]),geometry='geometry', crs=IFA.crs)
#             # neighbor_cells.sort_index(axis = 0) #sort by index
            
#             neighbor_cells['dist-start-cell'] = neighbor_cells.apply(lambda row: row['geometry'].distance(MPA_cellx), axis=1)
#             neighbor_cells.sort_values(by = 'dist-start-cell')
           
            
            
#             for idx, row in neighbor_cells.iterrows():
#                 if mpa_area  < Area_MPA : #(mpa_area + row['area']) < Area_MPA :
#                     mpa_area += row['area'] # increase area
#                     cells.drop([idx], inplace=True) # remove from cells

#                 # elif mpa_area > Area_MPA :
#                 else:
#                     neighbor_cells.drop([idx], inplace=True)

#             list_neighbor_cells = list(neighbor_cells['geometry']) # convert current neighbor cells to list
#             ALL_neighbor_cells += list_neighbor_cells # contains all neighbor cells over time
#             MPA_cell = unary_union(ALL_neighbor_cells) # unionize all neighbor cells over time
#             MPA_cell_shapefile = gpd.GeoDataFrame(geometry='geometry',columns=['geometry'],crs=IFA.crs) # empty geodataframe

#             if mpa_area >= Area_MPA:
#                 break

#         MPA = gpd.GeoDataFrame(ALL_neighbor_cells,geometry='geometry',columns=['geometry'], crs=IFA.crs) 
#         # MPA = gpd.overlay(MPA, IFA, how='intersection',keep_geom_type=True)
#         return MPA
#     # mpa_characteristics()

# #--------------------------------------------------------------------------------------------------------      
#     def MPA_Specifics():
#         global MPA, Area_MPA, mpa_area, neighbor_cells, MPA_cell_shapefile, MPA_cellx,  MPA_ConvexHull,  MPA_union

#         # only with single MPA
#         if Type_MPA == 'single' :
#             Area_MPA = Frac_MPA * (cells['area'].sum()) # total area of MPA as a fraction of IFA area
#             MPA = mpa_characteristics()
#             # ConvexHull = unary_union(MPA['geometry']).convex_hull
#             # MPA_union = gpd.GeoDataFrame([ConvexHull],geometry='geometry',columns=['geometry'],crs=IFA.crs)
#             # MPA_ConvexHull = gpd.overlay(IFA, MPA_union, how='intersection') # only convex cells within the IFA

#         # spaced MPA
#         elif Type_MPA == 'spaced' :
#             Area_MPA = (Frac_MPA * (cells['area'].sum())) / 2 # total area of two MPA as a fraction of IFA area /2
#             for i in range(2): 
#                 globals()[f'MPA{i+1}']  = mpa_characteristics() # dynamically name the variables
#             MPA =  gpd.GeoDataFrame(pd.concat([MPA1,MPA2]),geometry='geometry', crs=IFA.crs)
#             ConvexHull = unary_union(MPA['geometry']).convex_hull
#             MPA_union = gpd.GeoDataFrame([ConvexHull],geometry='geometry',columns=['geometry'],crs=IFA.crs)
#             MPA_ConvexHull = gpd.overlay(IFA, MPA_union, how='intersection') # only convex cells within the IFA

#         # no MPA
#         elif Type_MPA == 'nompa' : 
#             Area_MPA = 0
#             MPA = mpa_characteristics()

#     MPA_Specifics()
# #--------------------------------------------------------------------------------------------------------  

#     Regular_Cell() # reset cells 

# MPA_Characteristics()


In [ ]:
# def MPA_Characteristics():
#     global MPA, Area_MPA, mpa_area, neighbor_cells, MPA_cell_shapefile, MPA_cellx, MPA_ConvexHull, MPA_union, num_mpa 
# #--------------------------------------------------------------------------------------------------------    
    
    
#     def mpa_characteristics():
#         global MPA, Area_MPA, mpa_area, neighbor_cells, MPA_cell_shapefile, MPA_cellx,  MPA_ConvexHull,  MPA_union, cells_eez_12nm, num_mpa 
        
#         MPA_start_cell_index=random.randrange(len(cells_eez_12nm)) # index of closed fishing area start cell
#         MPA_cell =cells_eez_12nm.at[MPA_start_cell_index,'geometry'] # geometry of closed fishing area start cell
#         MPA_cellx = cells_eez_12nm.at[MPA_start_cell_index,'geometry'] # just for computing nearest distance to it
#         MPA_cell_shapefile = cells_eez_12nm.loc[MPA_start_cell_index : MPA_start_cell_index] # shapefile of closed fishing area start cell
#         mpa_area = 0 # initialize closed fishing area
#         ALL_neighbor_cells = []
#         # MPA_cell_shapefile = cells.loc[cells['geometry'].contains(IFA_poly.centroid)]
#         # MPA_cell = MPA_cell_shapefile.at[MPA_cell_shapefile.index.values[0],'geometry']
#         # Area_MPA = Frac_MPA * (cells['area'].sum()) # total area of MPA as a fraction of IFA area

#         while True:
#             # print(len(cells)) 
#             neighbor_cells = cells_eez_12nm.loc[ ((cells_eez_12nm['geometry']).distance(MPA_cell) < 5000) & (cells_eez_12nm['geometry'] != MPA_cell) \
#                                        # & ((cells_eez_12nm.intersection(MPA_cell)).geom_type == "LineString") 
#                                       ] # cells in neighborhood start mpa cell 
#             neighbor_cells =  gpd.GeoDataFrame(pd.concat([MPA_cell_shapefile,neighbor_cells]),geometry='geometry', crs=IFA.crs)
#             neighbor_cells['dist-start-cell'] = neighbor_cells.apply(lambda row: row['geometry'].distance(MPA_cellx), axis=1)
#             neighbor_cells.sort_values(by = 'dist-start-cell')
           
#             for idx, row in neighbor_cells.iterrows():
#                 if mpa_area  < Area_MPA :
#                     mpa_area += row['area'] 
#                     cells_eez_12nm.drop([idx], inplace=True) 
#                 else:
#                     neighbor_cells.drop([idx], inplace=True)

#             list_neighbor_cells = list(neighbor_cells['geometry']) # convert current neighbor cells to list
#             ALL_neighbor_cells += list_neighbor_cells # contains all neighbor cells over time
#             MPA_cell = unary_union(ALL_neighbor_cells) # unionize all neighbor cells over time
#             MPA_cell_shapefile = gpd.GeoDataFrame(geometry='geometry',columns=['geometry'],crs=IFA.crs) # empty geodataframe

#             if mpa_area >= Area_MPA:
#                 break

#         MPA = gpd.GeoDataFrame(ALL_neighbor_cells,geometry='geometry',columns=['geometry'], crs=IFA.crs) 
#         return MPA
    
#     # mpa_characteristics()
    
# #--------------------------------------------------------------------------------------------------------      
#     def MPA_Specifics():
#         global MPA, Area_MPA, mpa_area, neighbor_cells, MPA_cell_shapefile, MPA_cellx,  MPA_ConvexHull,  MPA_union, cells_eez_12nm, num_mpa 

# #         # only with single MPA
# #         if Type_MPA == 'single' :
# #             Area_MPA = Frac_MPA * (cells['area'].sum()) # total area of MPA as a fraction of IFA area
# #             MPA = mpa_characteristics()
            
# #         # spaced MPA
# #         elif Type_MPA == 'spaced' :
# #             num_mpa = 5
# #             Area_MPA = (Frac_MPA * (cells['area'].sum())) / num_mpa # total area of two MPA as a fraction of IFA area /2
# #             for i in range(num_mpa): 
# #                 globals()[f'MPA{i+1}']  = mpa_characteristics() # dynamically name the variables
# #             MPA =  gpd.GeoDataFrame(pd.concat([MPA1,MPA2,MPA3,MPA4,MPA5]),geometry='geometry', crs=IFA.crs)
# #             ConvexHull = unary_union(MPA['geometry']).convex_hull
# #             MPA_union = gpd.GeoDataFrame([ConvexHull],geometry='geometry',columns=['geometry'],crs=IFA.crs)
# #             MPA_ConvexHull = gpd.overlay(eez_12nm, MPA_union, how='intersection',keep_geom_type=False) # only convex cells within the IFA
     
# #         # no MPA
# #         elif Type_MPA == 'nompa' : 
# #             Area_MPA = 0
# #             MPA = mpa_characteristics()

         
#         num_mpa = 5 # nr. of hotspots
#         Area_MPA = (Frac_MPA * (cells['area'].sum())) / num_mpa # area of each hostspot
#         for i in range(num_mpa): 
#             globals()[f'MPA{i+1}']  = mpa_characteristics() # dynamically name the variables
#         MPA =  gpd.GeoDataFrame(pd.concat([MPA1,MPA2,MPA3,MPA4,MPA5]),geometry='geometry', crs=IFA.crs)
#         ConvexHull = unary_union(MPA['geometry']).convex_hull
#         MPA_union = gpd.GeoDataFrame([ConvexHull],geometry='geometry',columns=['geometry'],crs=IFA.crs)
#         MPA_ConvexHull = gpd.overlay(eez_12nm, MPA_union, how='intersection',keep_geom_type=False) # only convex cells within the IFA

#     MPA_Specifics()
# #--------------------------------------------------------------------------------------------------------  

#     Regular_Cell() # reset cells 

# MPA_Characteristics() 


In [ ]:
# def Fish_Initialize() :
#     global fish_geodata
    
#     center =[] # set the centers of clustering
#     xmin, ymin, xmax, ymax = IFA_poly.bounds # get the bounds of IFA
#     fish_geodata = gpd.GeoDataFrame(geometry='geometry',columns=['geometry'],crs=IFA.crs) # empty geodataframe

#     for cluster in range(num_clusters): # create center for clusters
#         while True:   
#             cluster_x = random.uniform(xmin, xmax)
#             cluster_y = random.uniform(ymin, ymax)
#             pnt = Point(cluster_x, cluster_y)
#             if IFA_poly.contains(pnt):
#                 center.append((cluster_x,cluster_y))
#                 break


#     for fish in range(init_fish): # create fishes around clusters
#         if (fish < 0.2 * init_fish) :
#             while True:   
#                 fish_x = random.gauss(center[0][0],point_deviation) # gaussian distribution around center
#                 fish_y = random.gauss(center[0][1],point_deviation)
#                 pnt = Point(fish_x, fish_y)
#                 if IFA_poly.contains(pnt):
#                     fish_geodata.at[fish,'geometry'] = pnt
#                     fish_geodata.at[fish,'x'] = fish_x
#                     fish_geodata.at[fish,'y'] = fish_y
#                     fish_geodata.at[fish,'agent type'] = 'fish'
#                     break

#         elif (0.20 * init_fish) <= fish < (0.4 * init_fish) :
#              while True:   
#                 fish_x = random.gauss(center[1][0],point_deviation)
#                 fish_y = random.gauss(center[1][1],point_deviation)
#                 pnt = Point(fish_x, fish_y)
#                 if IFA_poly.contains(pnt):
#                     fish_geodata.at[fish,'geometry'] = pnt
#                     fish_geodata.at[fish,'x'] = fish_x
#                     fish_geodata.at[fish,'y'] = fish_y
#                     fish_geodata.at[fish,'agent type'] = 'fish'
#                     break

#         elif (0.4 * init_fish) <= fish < (0.6 * init_fish) :
#             while True:   
#                 fish_x = random.gauss(center[2][0],point_deviation)
#                 fish_y = random.gauss(center[2][1],point_deviation)
#                 pnt = Point(fish_x, fish_y)
#                 if IFA_poly.contains(pnt):
#                     fish_geodata.at[fish,'geometry'] = pnt
#                     fish_geodata.at[fish,'x'] = fish_x
#                     fish_geodata.at[fish,'y'] = fish_y
#                     fish_geodata.at[fish,'agent type'] = 'fish'
#                     break

#         elif (0.6 * init_fish) <= fish < (0.8 * init_fish) :
#             while True:   
#                 fish_x = random.gauss(center[3][0],point_deviation)
#                 fish_y = random.gauss(center[3][1],point_deviation)
#                 pnt = Point(fish_x, fish_y)
#                 if IFA_poly.contains(pnt):
#                     fish_geodata.at[fish,'geometry'] = pnt
#                     fish_geodata.at[fish,'x'] = fish_x
#                     fish_geodata.at[fish,'y'] = fish_y
#                     fish_geodata.at[fish,'agent type'] = 'fish'
#                     break

#         else :
#             while True:   
#                 fish_x = random.gauss(center[4][0],point_deviation)
#                 fish_y = random.gauss(center[4][1],point_deviation)
#                 pnt = Point(fish_x, fish_y)
#                 if IFA_poly.contains(pnt):
#                     fish_geodata.at[fish,'geometry'] = pnt
#                     fish_geodata.at[fish,'x'] = fish_x
#                     fish_geodata.at[fish,'y'] = fish_y
#                     fish_geodata.at[fish,'agent type'] = 'fish'
#                     break

# Fish_Initialize() 


In [ ]:

# def Fish_Update() :
#     global fish_geodata
    
#     # randomly sample a geometry from the geodataframe
#     focal_fish_index=random.randrange(len(fish_geodata)) # index of focal fish
#     focal_fish=fish_geodata.at[focal_fish_index,'geometry'] 
    
#     # nearest point from exterior to the focal fish
#     ext=np.array(IFA_poly.geoms[1].exterior.coords) # coordinate of exterior
#     ext_multipoint = MultiPoint(ext) 
#     ext_nearest_geoms = nearest_points(focal_fish, ext_multipoint)
#     ext_coords_x = ext_nearest_geoms[1].x  
#     ext_coords_y = ext_nearest_geoms[1].y 
    
#     # computing zones around fish
#     repulsion_zone=focal_fish.buffer(rad_repulsion, cap_style = 1) # circle buffer the point
#     orientation_zone=focal_fish.buffer(rad_orientation, cap_style = 1) # circle buffer the point
#     attraction_zone=focal_fish.buffer(rad_attraction, cap_style = 1) # circle buffer the point

#     # locate fish within the zones of focal fish
#     repulsion = fish_geodata.loc[((fish_geodata['geometry']).within(repulsion_zone)) & ((fish_geodata['geometry']) != focal_fish)]
#     orientation = fish_geodata.loc[((fish_geodata['geometry']).within(orientation_zone)) & ((fish_geodata['geometry']) != focal_fish)]
#     attraction = fish_geodata.loc[((fish_geodata['geometry']).within(attraction_zone)) & ((fish_geodata['geometry']) != focal_fish)]
   
#     # if fishes within repulsion zone, move away from the spot that would be the center of mass (midpoint) of all  fish within repulsion zone
#     if len(repulsion) > 0: 
#         repulsion_x = repulsion['x'].mean()
#         repulsion_y = repulsion['y'].mean()
#         theta = (math.atan2((repulsion_y - focal_fish.y), (repulsion_x - focal_fish.x)) + math.pi ) % (2 * math.pi) # if greater than  (2 * math.pi) then compute with a minus
#         fish_geodata.at[focal_fish_index,'x'] +=  move_fish*math.cos(theta) 
#         fish_geodata.at[focal_fish_index,'y'] +=  move_fish*math.sin(theta) 
#         fish_geodata.at[focal_fish_index,'x'] = ext_coords_x if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'x']  # ( When fish-agent Outside a border of the IFA)
#         fish_geodata.at[focal_fish_index,'y'] = ext_coords_y if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'y']  # ( When fish-agent Outside a border of the IFA)
#         fish_geodata.at[focal_fish_index,'geometry'] =  Point(fish_geodata.at[focal_fish_index,'x'], fish_geodata.at[focal_fish_index,'y']) 

#     # if fishes within parallel-orientation zone, change direction to match the average direction of all the other fish  within parallel-orientation zone     
#     elif all([len(repulsion) == 0, len(orientation) > 0]):  
#         theta = (orientation.apply(lambda row: math.atan2((focal_fish.y - row['y']),(focal_fish.x - row['x'])) , axis=1)).mean()
#         fish_geodata.at[focal_fish_index,'x'] = math.cos(theta) 
#         fish_geodata.at[focal_fish_index,'y'] = math.sin(theta) 
#         fish_geodata.at[focal_fish_index,'x'] = ext_coords_x if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'x']  # ( When fish-agent Outside a border of the IFA)
#         fish_geodata.at[focal_fish_index,'y'] = ext_coords_y if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'y']  #  ( When fish-agent Outside a border of the IFA)
#         fish_geodata.at[focal_fish_index,'geometry'] =  Point(fish_geodata.at[focal_fish_index,'x'], fish_geodata.at[focal_fish_index,'y']) 

#     # if fishes within only the attraction zone, head towards the middle (midpoint) of the fishes in zone of attraction.   
#     elif all([len(repulsion) == 0, len(orientation) == 0, len(attraction) > 0]): 
#         attraction_x = attraction['x'].mean()
#         attraction_y = attraction['y'].mean()
#         theta = (math.atan2((attraction_y - focal_fish.y), (attraction_x - focal_fish.x)) + math.pi ) % (2 * math.pi) # if greater than  (2 * math.pi) then compute with a minus
#         fish_geodata.at[focal_fish_index,'x'] +=  move_fish*math.cos(theta) 
#         fish_geodata.at[focal_fish_index,'y'] +=  move_fish*math.sin(theta) 
#         fish_geodata.at[focal_fish_index,'x'] = ext_coords_x if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'x']  # ( When fish-agent Outside a border of the IFA) 
#         fish_geodata.at[focal_fish_index,'y'] = ext_coords_y if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'y']  # ( When fish-agent Outside a border of the IFA)
#         fish_geodata.at[focal_fish_index,'geometry'] =  Point(fish_geodata.at[focal_fish_index,'x'], fish_geodata.at[focal_fish_index,'y']) 

#     # if no fishes in all the zone, move in a random direction 
#     elif all([len(repulsion) == 0, len(orientation) == 0, len(attraction) == 0]):  
#         theta = 2*math.pi*random.random()  
#         fish_geodata.at[focal_fish_index,'x'] +=  move_fish*math.cos(theta) 
#         fish_geodata.at[focal_fish_index,'y'] +=  move_fish*math.sin(theta)
#         fish_geodata.at[focal_fish_index,'x'] = ext_coords_x if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'x']  # ( When fish-agent Outside a border of the IFA) 
#         fish_geodata.at[focal_fish_index,'y'] = ext_coords_y if IFA_poly.contains(Point(fish_geodata.at[focal_fish_index,'x'],fish_geodata.at[focal_fish_index,'y'])) == False else fish_geodata.at[focal_fish_index,'y']  # ( When fish-agent Outside a border of the IFA)
#         fish_geodata.at[focal_fish_index,'geometry'] =  Point(fish_geodata.at[focal_fish_index,'x'], fish_geodata.at[focal_fish_index,'y']) 

#     # logistid reproduction
#     if random.random() <   growth_prob * (1 - len(fish_geodata) / K):
#         fish_geodata.loc[len(fish_geodata)] = [Point(focal_fish.x , focal_fish.y), focal_fish.x , focal_fish.y, 'fish' ]
        
            
# Fish_Update() 


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Parameters
# r = 0.693          # intrinsic growth rate
# K = 2000         # carrying capacity
# day = np.arange(0, 91)  # days from 0 to 90
# N_values = np.linspace(0, K, 2000)  # population values

# # mu = 45       # Centred at day 45
# # sigma = 15  # Standard deviation

# mu = 45       # peak day of upwelling (run for three months (July-September), 90 days, peak at mid-day of August
# # x_start = 0
# # x_end= 90
# # target_value = 0.5 # of x_start and x_end
# # sigma = abs(x_start - mu) / np.sqrt(-2 * np.log(target_value))
# sigma =15


# # Logistic reproduction probability normalised
# def logistic_prob(N):
#     max_repro = r * K / 4  # maximum at N = K/2
#     return (r * N * (1 - N / K)) / max_repro

# # Gaussian upwelling intensity centred at day 45
# def upwelling_intensity(day):
#     gaussian_values = np.exp(-0.5 * ((day - mu) / sigma)**2)
#     gaussian_values /= np.max(gaussian_values)   # Normalize values to range [0, 1]
#     return gaussian_values

# # Compute logistic probabilities
# logistic_probs = logistic_prob(N_values)

# # Compute upwelling intensities for each day
# upwelling_probs = upwelling_intensity(days)

# # Create a meshgrid for population and days
# day_grid, N_grid = np.meshgrid(days, N_values)

# # Compute combined reproduction probability
# combined_prob = logistic_prob(N_grid) * upwelling_intensity(day_grid)

# plt.clf
# # Plotting
# plt.figure(figsize=(10, 6))
# contour = plt.contourf(day_grid, N_grid, combined_prob, levels=50, cmap='viridis')
# plt.colorbar(contour, label='Combined probability of reproduction')
# plt.xlabel('Day')
# plt.ylabel('Population Size')
# plt.title('Combined Effect of logistic growth and upwelling intensity')
# plt.axvline(45, color='red', linestyle='--', label='Peak Upwelling (Day 45)')
# plt.axhline(K/2, color='blue', linestyle='--', label='Peak Logistic Reproduction (K/2)')
# plt.legend()
# plt.tight_layout()
# plt.savefig('reproduction_prob.pdf', bbox_inches='tight', pad_inches=0.1, dpi=1000) 
# plt.show()

In [ ]:
# def Pirogue_Initialize() :
#     global pirogue_geodata, cells_pirogue_movement

#     xmin, ymin, xmax, ymax = IFA_poly.bounds # get the bounds of IFA
#     pirogue_geodata = gpd.GeoDataFrame(geometry='geometry',columns=['geometry'],crs=IFA.crs) # empty geodataframe
#     cells_pirogue_movement = gpd.overlay(cells, MPA_ConvexHull, how='difference') # cells a pirogue can move to - LARGE_CLOSURE
#     # cells_pirogue_movement = gpd.overlay(cells, MPA, how='difference') # cells a pirogue can move to - INDIVIDUAL_CLOSURE
#     # cells_pirogue_movement = gpd.overlay(cells, cells, how='intersection',keep_geom_type=True) # cells a pirogue can move to - NO_CLOSURE
#     # cells_pirogue_movement = gpd.GeoDataFrame(geometry='geometry',columns=['geometry'],crs=IFA.crs) # cells a pirogue can move to - FULL_CLOSURE
   

#     # setting the charactersitics of pirogues
#     for pirogue in range(num_pirogue): 
#         while True:   
#             pirogue_x = random.uniform(xmin, xmax)
#             pirogue_y = random.uniform(ymin, ymax)
#             pnt = Point(pirogue_x,pirogue_y)
#             if len(cells_pirogue_movement.loc[cells_pirogue_movement['geometry'].contains(pnt)]) > 0: 
#                 pirogue_geodata.at[pirogue,'geometry'] = pnt
#                 pirogue_geodata.at[pirogue,'x'] = pirogue_x
#                 pirogue_geodata.at[pirogue,'y'] = pirogue_y
#                 pirogue_geodata.at[pirogue,'agent type'] = 'pirogue'
#                 pirogue_geodata.at[pirogue,'catch'] = 0
#                 pirogue_geodata.at[pirogue,'current catch'] = 0
#                 pirogue_geodata.at[pirogue,'cooperative-trait'] = 'coop' if  pirogue <  int(frac_coop * num_pirogue) else  'non-coop'          # set their cooperative-trait
#                 pirogue_geodata.at[pirogue,'effort'] = coop_effort if  pirogue <  int(frac_coop * num_pirogue) else noncoop_effort          # set their cooperative-trait
#                 pirogue_geodata.at[pirogue,'time0'] = 0
#                 break
                
# Pirogue_Initialize()


In [ ]:
# def Pirogue_Update() :
#     global pirogue_geodata, repulsion, focal_pirogue_cell ,focal_pirogue_cell_fishes, focal_pirogue_neighbor_cells
#     global focal_pirogue_cell, focal_pirogue, exploration_poly_shapefile, union_neighborhood, main_exploration_poly_shapefile

#     xmin, ymin, xmax, ymax = IFA_poly.bounds # get the bounds of IFA

#     #randomly sample a geometry from the geodataframe
#     focal_pirogue_index=random.randrange(len(pirogue_geodata)) # index of focal pirogue
#     focal_pirogue=pirogue_geodata.loc[focal_pirogue_index:focal_pirogue_index]  # sample focal piorgue using the index
    
#     # # # cell containing the focal_pirogue
#     focal_pirogue_cell = cells_pirogue_movement.loc[cells_pirogue_movement['geometry'].contains(focal_pirogue.at[focal_pirogue_index,'geometry'])]
    
#     # cells containing neighbors (moore neighborhood) of focal pirogue  
#     focal_pirogue_neighbor_cells = cells_pirogue_movement.loc[ ((cells_pirogue_movement['geometry']).distance(focal_pirogue_cell.at[focal_pirogue_cell.index[0],'geometry']) == 0)  &  (cells_pirogue_movement['geometry'] != focal_pirogue_cell.at[focal_pirogue_cell.index[0],'geometry']) ]
   
#     if len(focal_pirogue_neighbor_cells) > 0 : # if focal cell has neighbor cells
#         # find all pirogues in the focal_pirogue_neighbor_cells
#         focal_pirogue_neighbors=intersect_using_spatial_index(source_gdf=pirogue_geodata, intersecting_gdf=focal_pirogue_neighbor_cells)
#     else:
#         focal_pirogue_neighbors = gpd.GeoDataFrame(geometry='geometry',columns=['geometry'],crs=IFA.crs)
        
    
#     # exploration area
#     exploration_poly =   ((focal_pirogue_cell.at[focal_pirogue_cell.index[0],'geometry']).centroid).buffer(math.sqrt(exploration_area)/2 , cap_style = 3) # square polygon of area to explore (buffered at focal pirogue)
#     exploration_poly_shapefile =  gpd.GeoDataFrame([exploration_poly],geometry='geometry',columns=['geometry'], crs=IFA.crs) # square polygon to shapefile
#     exploration_poly_shapefile=gpd.overlay(cells_pirogue_movement, exploration_poly_shapefile, how='intersection',keep_geom_type=True) # take only the exploration area within pirogue movement area
#     if len(focal_pirogue_neighbor_cells ) > 0:
#         union_neighborhood = gpd.overlay(focal_pirogue_neighbor_cells, focal_pirogue_cell, how='union',keep_geom_type=True) # union of moore neighborhood (i.e. including focal pirogue cell) ,keep_geom_type=True   
#     else:
#         union_neighborhood = focal_pirogue_cell
#     main_exploration_poly_shapefile = gpd.overlay(exploration_poly_shapefile, union_neighborhood ,how='difference',keep_geom_type=True) # take only exploration area excluding moore neighborhood
     
        
#     # EEI algorithm    
#     Xmin, Ymin, Xmax, Ymax = main_exploration_poly_shapefile.total_bounds
#     if random.random() <  exploration_prob: # explore
#         while True:   
#             exploration_x = random.uniform(Xmin, Xmax)
#             exploration_y = random.uniform(Ymin, Ymax)
#             pnt = Point(exploration_x, exploration_y)
#             pnt_shapefile =  gpd.GeoDataFrame([pnt],geometry='geometry',columns=['geometry'], crs=IFA.crs) 
#             pnt_within_explore=intersect_using_spatial_index(source_gdf=pnt_shapefile, intersecting_gdf=main_exploration_poly_shapefile) # check if explore point is within main_exploration_poly_shapefile
#             if len(pnt_within_explore) > 0 :
#                 pirogue_geodata.at[focal_pirogue_index,'geometry'] = pnt
#                 pirogue_geodata.at[focal_pirogue_index,'x'] = exploration_x 
#                 pirogue_geodata.at[focal_pirogue_index,'y'] = exploration_y
#                 break

#     else : # imitate / exploit
#         if len(focal_pirogue_neighbors) > 0 :
#             idx_maxcatch_focal_pirogue_neighbor=focal_pirogue_neighbors['current catch'].idxmax() # index of focal_pirogue_neighbor with max current catch
#             if (pirogue_geodata.at[idx_maxcatch_focal_pirogue_neighbor,'current catch']) > (pirogue_geodata.at[focal_pirogue_index,'current catch']):
#                 pirogue_geodata.at[focal_pirogue_index,'geometry'] = pirogue_geodata.at[idx_maxcatch_focal_pirogue_neighbor,'geometry']
#                 pirogue_geodata.at[focal_pirogue_index,'x'] =  pirogue_geodata.at[idx_maxcatch_focal_pirogue_neighbor,'x']
#                 pirogue_geodata.at[focal_pirogue_index,'y'] =   pirogue_geodata.at[idx_maxcatch_focal_pirogue_neighbor,'y']


#     # harvest fishes from focal pirogue cell
#     # focal_pirogue_cell_fishes=intersect_using_spatial_index(source_gdf=fish_geodata, intersecting_gdf=focal_pirogue_cell) # fishes in focal pirogue cell
#     focal_pirogue_cell_fishes=intersect_using_spatial_index(source_gdf=fish_geodata, intersecting_gdf= union_neighborhood) # fishes in focal pirogue cell + neigboring cell
#     focal_pirogue_catch = int(catchability * pirogue_geodata.at[focal_pirogue_index,'effort'] * len(focal_pirogue_cell_fishes)) # number of fishes to harvest
#     pirogue_geodata.at[focal_pirogue_index,'current catch'] += focal_pirogue_catch # add harvest to current catch 
#     pirogue_geodata.at[focal_pirogue_index,'catch'] += focal_pirogue_catch  # add harvest to total catch 
#     fish_index_remove=list(focal_pirogue_cell_fishes.sample(n=focal_pirogue_catch).index.values) # index of fishes to remove
#     fish_geodata.drop(fish_index_remove, inplace=True) # remove the fish
#     fish_geodata.reset_index(drop=True, inplace=True)  # reset the index of geodataframe
        
    
# Pirogue_Update() 


In [ ]:
# #randomly sample a pirogue
# focal_pirogue_index = 12 # index of focal pirogue
# focal_pirogue=pirogue_geodata.loc[focal_pirogue_index:focal_pirogue_index]  # sample focal piorgue using the index

# # ## cell containing the focal_pirogue
# focal_pirogue_cell = cells_pirogue_movement_all.loc[cells_pirogue_movement_all['geometry'].contains(focal_pirogue.at[focal_pirogue_index,'geometry'])]
# # focal_pirogue_neighbors=intersect_using_spatial_index(source_gdf=pirogue_geodata, intersecting_gdf=focal_pirogue_cell) # find all pirogues in the focal pirogue cell
# # neighbors_without_focal_pirogue = focal_pirogue_neighbors.loc[focal_pirogue_neighbors['geometry'] != focal_pirogue.at[focal_pirogue_index,'geometry'] ] # neighbor pirogues without focal pirogue

# # # Moore neighborhood cells of focal pirogue cell
# cells_pirogue_movement_all['touch1']=cells_pirogue_movement_all.apply(lambda row: row['geometry'].intersects(focal_pirogue_cell['geometry']), axis=1) 
# focal_pirogue_cell_neighbors=cells_pirogue_movement_all.loc[cells_pirogue_movement_all['touch1'] == True]

# moore_neighbors=intersect_using_spatial_index(source_gdf=pirogue_geodata, intersecting_gdf=focal_pirogue_cell_neighbors) 
# moore_maxcatch_idx=moore_neighbors['current catch'].idxmax()  
# moore_maxcatch_geom = moore_neighbors.loc[moore_maxcatch_idx, 'geometry']
# moore_maxcatch_cell=cells_pirogue_movement_all.loc[cells_pirogue_movement_all['geometry'].contains(moore_maxcatch_geom)] 


In [ ]:
# def Hotspots_upwelling():
#     global upwelling_gdf_bound, cells_without_upwelling_gdf
    
#     #Define polygon boundary for Central Upwelling Zone
#     # upwelling_coords = [(-3.000, 4.500), (-3.000, 5.500), (-0.500, 5.500), (-0.500, 4.500), (-3.000, 4.500)]
#     # upwelling_polygon = Polygon(upwelling_coords)
#     # Define bounding box coordinates for the central upwelling zone
#     lat_min = 4.6
#     lat_max = 5.4
#     lon_min = -3.1
#     lon_max = -1.7
#     # Create a GeoDataFrame for the upwelling zone rectangle
#     upwelling_geom = box(lon_min, lat_min, lon_max, lat_max)
#     upwelling_gdf = gpd.GeoDataFrame({'Zone': ['Central Upwelling Zone']}, geometry=[upwelling_geom], crs="EPSG:4326") # the crs coordinate with the given. coordinates
#     upwelling_gdf  = upwelling_gdf.to_crs(epsg=32630)
#     upwelling_gdf_bound = gpd.overlay(cells, upwelling_gdf, how='intersection') # boundary of central upwelling area within the IFA
#     upwelling_gdf_bound['area'] = upwelling_gdf_bound.apply(lambda row: row['geometry'].area, axis=1)
    
#     # assign temperatures based on zones (hotspot, central upwelling, none)
#     cells['temperature'] = cells.apply(lambda row: random.uniform(20, 22) if any(row['geometry'].within(row1) or row['geometry'].intersects(row1) or row['geometry'].touches(row1) for row1 in upwelling_gdf_bound['geometry']) # hotspots temp
#                                        else random.uniform(22, 26), 
#                                        axis=1)
    
#     # upwelling_gdf_without_hotspot_gdf = gpd.overlay(upwelling_gdf_bound, hotspot_gdf, how='difference') # boundary of central upwelling area within the IFA without hotspots
#     cells_without_upwelling_gdf = gpd.overlay(cells, upwelling_gdf, how='difference') # boundary of central upwelling area within the IFA
  
    
    
#     # #Define coordinates for Sardinella spawning hotspots
#     # hotspot_coords = {
#     #     "Elmina": (-1.350, 5.083),
#     #     "Moree": (-1.450, 5.100),
#     #     "Apam": (-0.733, 5.283),
#     #     "Axim": (-2.233, 4.866)
#     #     }
#     # # Create GeoDataFrame for Sardinella spawning hotspots
#     # hotspot_points = [Point(lon, lat) for lon, lat in hotspot_coords.values()]
#     # hotspot_gdf = gpd.GeoDataFrame({'Town': list(hotspot_coords.keys())}, geometry=hotspot_points, crs="EPSG:4326")
#     # hotspot_gdf  = hotspot_gdf.to_crs(epsg=32630)
#     # hotspot_gdf['geometry'] = hotspot_gdf.apply(lambda row: box(row['geometry'].x, row['geometry'].y, row['geometry'].x-25000, row['geometry'].y-25000 ), axis=1) # set length of square hotspot to 25,000 meters (i.e., area of 625 square kilometers) 
#     # hotspot_gdf['area'] = hotspot_gdf.apply(lambda row: row['geometry'].area, axis=1)
#     # upwelling_gdf_without_hotspot_gdf = gpd.overlay(upwelling_gdf_bound, hotspot_gdf, how='difference') # boundary of central upwelling area within the IFA without hotspots

#     # # assign temperatures based on zones (hotspot, central upwelling, none)
#     # cells['temperature'] = cells.apply(lambda row: random.uniform(2, 5) if any(row['geometry'].within(row1) or row['geometry'].intersects(row1) or row['geometry'].touches(row1) for row1 in hotspot_gdf['geometry']) # hotspots temp
#     #                                else random.uniform(6, 10) if any(row['geometry'].within(row1) or row['geometry'].intersects(row1) or row['geometry'].touches(row1) for row1 in upwelling_gdf_without_hotspot_gdf['geometry']) # central upwelling temp
#     #                                else random.uniform(10, 20), 
#     #                                axis=1)

#     # fig, ax = plt.subplots(figsize=(16,8))
#     # cells.plot(ax=ax, column='temperature', cmap='inferno', legend=True, edgecolor='black', lw=1, alpha=1,
#     #            legend_kwds={
#     #                 'shrink': 0.6,         # Shrinks the colorbar vertically
#     #                 'aspect': 20,          # Controls the thickness of the colorbar
#     #                 'orientation': 'vertical',  # Can also be 'horizontal'
#     #                'label': 'Temperature (°C)'
#     #                 })
#     # # cells.plot(ax=ax, facecolor='white', edgecolor='black', lw=1)
#     # upwelling_gdf_bound.plot(ax=ax,facecolor='none', edgecolor='blue', lw=1)
#     # # #fish_geodata.plot(ax=ax,facecolor='red', edgecolor='none')
#     # # # #pirogue_geodata.plot(ax=ax,facecolor='black', edgecolor='none')

# Hotspots_upwelling()


## Figure of fishing ground configuration for proposal

In [ ]:
# eez_12nm_ivorycoast  = gpd.read_file(os.path.join(os.getcwd(), 'eez_12nm_ivorycoast/eez_12nm.shp')) # Exclusive Economic Zone (200 nautical miles)
# # reproject crs 
# eez_12nm_ivorycoast= eez_12nm_ivorycoast.to_crs(epsg=25000)



# #Ghana land
# place_name = 'Ghana'
# boundaries  = ox.geocode_to_gdf(place_name)
# boundaries = boundaries.to_crs(IFA.crs)
# #Ivory coast
# place_name1 = 'Ivory coast'
# boundaries1  = ox.geocode_to_gdf(place_name1)
# boundaries1 = boundaries1.to_crs(IFA.crs)

# #land
# a = gpd.overlay(cell, boundaries, how='intersection')
# b = gpd.overlay(cells_eez_12nm, a, how='symmetric_difference') # ghana land 
# #ocean
# c = gpd.overlay(cells_eez_24nm, a, how='union')
# d = gpd.overlay(cell, c, how='symmetric_difference')
# f = gpd.overlay(boundaries1, d,how='intersection') # ivory coast 
# g = gpd.overlay(d, f, how='symmetric_difference') # atlantic ocean 

# #eez_12nm_ivorycoast.plot()
# n = gpd.overlay(eez_12nm_ivorycoast, f, how='intersection')
# #############################################################################################


# fig = plt.figure(figsize=(35, 15))
# # fig.suptitle(r'Fishing Ground Configuration for Closed Fishing Season', fontsize=20)
# fig.subplots_adjust(hspace=0.05, wspace=0) # horizontal and vertical space hspace =0.2, wspace=0.05

# for i in range(3): 
#     ax = fig.add_subplot(3,1,i+1)
    
#     if i == 0:
#         minx1, miny1, maxx1, maxy1 = IFA['geometry'].total_bounds# ezz bounds

#         # create a legend: we'll plot empty lists with the desired color, label, symbol
#         for facecol, label, edgecol, symb, alph in [('white','Inshore Fishing Area','black','s', 1), 
#                                           ('white','Spawning Hotspot','mediumblue','s', 1),
#                                           ('red','Closed to Fishing ','red','s', 0.3),
#                                           ('black','Fish Agent','black','.', 1) ,
#                                           ('white','Fishing Agent','black','o', 1),
#                                          # ('grey','Ghana Land','none','s', 1),
#                                           #('skyblue','Atlantic Ocean','none','s', 1)
#                                                    ]:
#             ax.scatter([], [], facecolor=facecol, s=100, label=label, alpha=alph, edgecolors=edgecol, marker=symb,linewidths=1.5 )
#             ax.legend(facecolor="white", edgecolor="none",prop={"size":12}, loc=(0.15,0.53),ncol=1 ) #loc=(0.7,0.02)


#     # plot IFA, cells, MPA, fish, pirogue
#     IFA.plot(ax=ax,facecolor='white', edgecolor='black', lw=1,zorder=1,alpha=0.5)
#     cells.plot(ax=ax,facecolor="none", edgecolor='black', lw=0.4,zorder=1, alpha=0.5)
#     b.plot(ax=ax,facecolor='grey', edgecolor='none', lw=1,zorder=1,alpha=0.5)
#     g.plot(ax=ax,facecolor='skyblue', edgecolor='none', lw=1,zorder=1,alpha=0.5)
#     n.plot(ax=ax,facecolor='skyblue', edgecolor='none', lw=1,zorder=1,alpha=0.5)
#     #f.plot(ax=ax,facecolor='black', edgecolor='none', lw=1,zorder=1,alpha=0.5)
#     if len(MPA) > 0 :
#         if i == 0:
#             MPA.plot(ax=ax,facecolor='none', edgecolor='blue', lw=1.4,zorder=2, alpha=0.5) # Plot MPA
#             cells.plot(ax=ax,facecolor='red', edgecolor='none', lw=1.4,zorder=3, alpha=0.15) # Plot MPA
        
#         elif i == 1:
#             MPA.plot(ax=ax,facecolor='none', edgecolor='blue', lw=1.4,zorder=2, alpha=0.5) # Plot MPA
#             MPA.plot(ax=ax,facecolor='red', edgecolor='none', lw=1.4,zorder=3, alpha=0.17) # Plot MPA
#             pirogue_geodata.plot(ax=ax,facecolor='none', edgecolor='black' ,marker = 'o', linewidth =0.5,markersize=10,zorder=2)
#         else:
#             MPA.plot(ax=ax,facecolor='none', edgecolor='blue', lw=1.4,zorder=2, alpha=0.5) # Plot MPA
#             # convex hull around mpa
#             MPA_ConvexHull.plot(ax=ax,facecolor='red', edgecolor='none',lw=1.4, zorder=3,alpha=0.17)
#             pirogue_geodata.plot(ax=ax,facecolor='none', edgecolor='black' ,marker = 'o', linewidth =0.5,markersize=10,zorder=2)

            
#     fish_geodata.plot(ax=ax,color='black', edgecolor='black' ,marker = '.',markersize=4,zorder=2)
#     # pirogue_geodata.plot(ax=ax,facecolor='none', edgecolor='black' ,marker = 'o', linewidth =0.5,markersize=10,zorder=2)


#     # ax.set_xlabel('X Coordinates ($10^3$ meters)',fontsize=15)
#     ax.set_ylabel('Y Coordinates (km)',fontsize=15)
#     if i > 1:
#         ax.set_xlabel('X Coordinates (km)',fontsize=15)
#     ax.set_ylabel('Y Coordinates (km)',fontsize=15)
    
#     if i ==0 :
#         #ax.text(50 *1000, 130 * 1000, 'Full closure of inshore fishing area',fontsize=15)
#         ax.text(35 *1000, 145 * 1000, 'A',fontsize=22)
#         ax.text(300 *1000, 130 * 1000, 'Ghana ',fontsize=20)
#         ax.text(350 *1000, -2 * 1000, 'Atlantic Ocean ',fontsize=20)
#         #ax.set_title("Ghana's Inshore Fishing Area : spatial dynamics of fish and fisher agents",fontsize=15) # title
#         ax.set_xticks([]) 
        
#     elif i ==1 :
#         #ax.text(50 *1000, 130 * 1000, 'Individual closures of sardinella spawning hotspots',fontsize=15)
#         ax.text(35 *1000, 145 * 1000, 'B',fontsize=22)
#         ax.set_xticks([]) 
              
#     else :
#         #ax.text(50 *1000, 130 * 1000, 'Large closure covering all sardinella  spawning hotspots',fontsize=16)
#         ax.text(35 *1000, 145 * 1000, 'C',fontsize=22)
        
#     # set axis limit as boundaries of Ghana
#     ax.set_xlim(([minx1, maxx1]))
#     ax.set_ylim(([miny1, maxy1])) 
#     # ax.set_yticks([]) ; ax.set_xticks([]) 
#     # ax.set_axis_off()

#     # make axis number label not scientific notation
#     for axis in [ax.xaxis, ax.yaxis]:
#         formatter = ScalarFormatter()
#         formatter.set_scientific(False)
#         axis.set_major_formatter(formatter)
    
# #     # # manually normalise to remove negative numbers
# #     y_normalize = lambda y, _: f'{y+25000:g}'
# #     ax.yaxis.set_major_formatter(y_normalize)
# #     x_normalize = lambda x, _: f'{x+25000:g}'
# #     ax.xaxis.set_major_formatter(x_normalize)
    
#     # change axis from meters to kilometers
#     #m2km = lambda x, _: f'{ round((x + abs(miny1)) / 1000, 0) :g}'
#     m2km = lambda x, _: f'{ round((x) / 1000 + 25, 0) :g}'
#     ax.yaxis.set_major_formatter(m2km)
#     ax.xaxis.set_major_formatter(m2km)
    


#     # plt.tight_layout()  
#     plt.savefig('FIG.pdf', bbox_inches='tight', pad_inches=0.02, dpi=1000)
    